In [1]:
#process the quantconnect notebook by sending a cell code, and reading response

from selenium import webdriver
from selenium.common.exceptions import NoSuchWindowException,WebDriverException,StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import datetime as dt
from datetime import timedelta

import pandas as pd

import time as tm
import json
import pickle
import pprint
from mt_utils import mt_dict_to_df,mt_df_to_dict

from math import nan

#from mt_data import options_cache,concat_dfs,add_underlying_cache,search_underlying_cache,add_options_cache



In [2]:
def wait_for_element(func,id,tries):
    while True:
        try:
            element = func(id)
            return element
        
        except NoSuchElementException as e:
            #print("except 1")
            tries -= 1
            if not tries:
                print("raise 5")
                raise e
            tm.sleep(.1)
            
def wait_minutes(count):
    while count:
        print("waiting {} minutes".format(count))
        tm.sleep(60)
        count-=1

def manual_login(driver,qc_info):

    log_in = wait_for_element(driver.find_element_by_xpath,'//a[@href="/login"]',10)

    log_in.click()
    
    social_group = wait_for_element(driver.find_element_by_class_name,'btn-social-group',10)
    
    buttons = social_group.find_elements_by_class_name('btn')
    buttons[1].click()
    
    fb_login_form = wait_for_element(driver.find_element_by_id,'loginform',10)
    
    fb_email = fb_login_form.find_element_by_id('email')
    fb_email.send_keys(qc_info['email'])
    fb_pass = fb_login_form.find_element_by_id('pass')
    fb_pass.send_keys(qc_info['password'])
    fb_login = fb_login_form.find_element_by_id('loginbutton')
    fb_login.click()
    wait_for_element(driver.find_element_by_class_name,'start-here-section',10)
    return

def load_research_notebook(driver):
    while True:
        file_list = wait_for_element(driver.find_elements_by_class_name,"file-name",10)
        if not len(file_list):
            tm.sleep(.1)
        else:
            break
            
    for f in file_list:
        if f.text == 'research.ipynb':
            print("clicking",f.text)
            f.click()
            break
            
    notebook_frame = wait_for_element(driver.find_element_by_class_name,"notebook-frame",10)        
    driver.switch_to.frame(notebook_frame)
            
def load_project(driver,info):
    driver.get(info['url_project'])
    
def datetime_as_args(date):
    return '{},{},{},{},{}'.format(date.year,date.month,date.day,date.time().hour,date.time().minute)

def send_combo_keys(driver,key1,key2):
    actions = ActionChains(driver)
    actions.key_down(key1)
    actions.send_keys(key2)
    actions.key_up(key1)

def send_multiline_text(driver,element,text_lines):
    text_area = element.find_element_by_tag_name("textarea")
    for line in text_lines:
        send_combo_keys(driver,Keys.ALT,Keys.ARROW_LEFT) # go to beginning of line in case of auto indention
        text_area.send_keys(line+ Keys.ENTER)  #send text

class Error(Exception):
    pass

class QcResponseError(Error):
    def __init__(self,error_word):
        self.error_string = error_word
        
class MissingStrikeError(Error):
    def __init__(self,eq_string,lt_string,gt_string):
        self.eq_strike = eq_string
        self.lt_strike = lt_string
        self.gt_strike = gt_string

class MissingExpireError(Error):
    def __init__(self,eq_string,lt_string,gt_string):
        self.eq_expire = eq_string
        self.lt_expire = lt_string
        self.gt_expire = gt_string

    pass

def get_cell(cell_num):
    cells = driver.find_elements_by_class_name("cell")

    cell = cells[cell_num]
    
    return cell


def get_code_mirror(cell):
    
    code_mirror = cell.find_element_by_class_name("CodeMirror")
    
    return code_mirror


def execute_cell(cell):
    
    get_code_mirror(cell).find_element_by_tag_name("textarea").send_keys(Keys.SHIFT + Keys.ENTER)   #execute cell with SHIFT_ENTER
    
def enter_cell(cell):
    
    code_mirror = get_code_mirror(cell)

    code_mirror.click()
    
    return code_mirror

 

def execute_cell_lines(cell,cell_lines):
    
    
    enter_cell(cell)
    
    code_mirror = get_code_mirror(cell)
    

    code_mirror.find_element_by_tag_name("textarea").send_keys(Keys.CONTROL + 'a')  #select all text in cell so it will be replaced
    send_multiline_text(driver,code_mirror,cell_lines)

    
    execute_cell(cell)
    

def get_cell_response(cell):
    
    output_subarea = wait_for_element(cell.find_element_by_class_name,"output_subarea",10000)  #read results
    
    return output_subarea.text

def get_parsed_cell_response(cell,tries = 200):

    while(tries):
        text = get_cell_response(cell)
        
        error = find_error(text)
        if error is not None:
            words = error.split()
            print("-----------------------------------found error:",error)
            if words[1] == 'MissingStrike':
                raise MissingStrikeError(words[2],words[3],words[4])
            if words[1] == 'MissingExpire':
                raise MissingExpireError(words[2],words[3],words[4])
            if words[0].find("Exception") >= 0:

                raise QcResponseError(words[0])
            print("reached here 1")
            return None
        
        start_index = find_start_line_sequence(text,'[')
        if start_index != None:
            return text[start_index:]
        
        tm.sleep(.2)
        tries -= 1
        
    return None

def find_start_line_sequence(text,start_char):
    last_char = '\n'
    
    for i,c in enumerate(text):
        
        if last_char == '\n' and c == start_char:
            return i
        
        last_char = c
        
    return None

def find_error(text):
    for line in text.splitlines():
#        if len(line) < 2000:
#            print("find_error",line)
        words = line.split()
        if len(words):
            if words[0] == 'Error:':
                print("found error",line)
                return line
        if words[0].find('Exception') >=0:
            return line
        
    return None

def execute_cell_code(cell_lines):
    cell = get_cell(1)

    execute_cell_lines(cell,cell_lines)
    
    output_text = get_parsed_cell_response(cell)
    
    if output_text:
    
        results = eval(output_text)
            
        df = mt_dict_to_df(results)
        
        if len(df):
            df['expiry'] = [pd.to_datetime(ts) for ts in df['expiry']]
        
        return df
    else:
        print("raise 2")
        raise QcResponseError("Timed out, no response")
        

def get_option(sym,start_time,end_time,strike_date,spread_type,strike):
    print("get_option:",sym,start_time,end_time,spread_type,strike)
    time_format = "%Y,%m,%d,%H,%M"
    cell_lines = [
        "import datetime as dt",
        "#from mt_utils import get_option_data",
        "#from mt_utils import df_to_dict",
        "qb = QuantBook()",
        "#df = get_option_data(qb,'{}',dt.datetime({}),dt.datetime({}),dt.datetime({}),'{}',{},Resolution.Minute)".format(
            sym,
            datetime_as_args(start_time),
            datetime_as_args(end_time),
            datetime_as_args(strike_date),
            spread_type,
            strike),

        "df = get_option_data(qb,'{}',dt.datetime({}),dt.datetime({}),dt.datetime({}),'{}',{})".format(
            sym,
            datetime_as_args(start_time),
            datetime_as_args(end_time),
            datetime_as_args(strike_date),
            spread_type,
            strike),
        "dct = df_to_dict(df)",
        "print(dct)"
    ]
    
    df = execute_cell_code(cell_lines)
    
    return df

def get_history(symbol,start_time,end_time,keep_list = []):
    time_format = "%Y,%m,%d,%H,%M"

    cell_lines = [
            "import datetime as dt",
            "qb = QuantBook()",
            "equity = qb.AddEquity('{}')".format(symbol),
            "start_time = dt.datetime({})".format(datetime_as_args(start_time)),
            "end_time = dt.datetime({})".format(datetime_as_args(end_time)),
            "history = qb.History(equity.Symbol, start_time, end_time)",
            "history.reset_index(inplace = True)",
            "#print('columns: ',history.columns)",
            "keep_columns(history,{})".format(keep_list),
            "dct = df_to_dict(history)",
            "print(dct)"]
    
    df = execute_cell_code(cell_lines)
    
    return df

def get_futures(symbol,start_time,end_time,keep_list = []):
    time_format = "%Y,%m,%d,%H,%M"
    
    cell_lines = [
            "import datetime as dt",
            "qb = QuantBook()",
            "es = qb.AddFuture('{}')".format(symbol),
            "#es = qb.AddFuture(Futures.Indices.SP500EMini)",
            "es.SetFilter(timedelta(0), timedelta(100))",
            "start_time = dt.datetime({})".format(datetime_as_args(start_time)),
            "end_time = dt.datetime({})".format(datetime_as_args(end_time)),
            "future_history = qb.GetFutureHistory(es.Symbol,start_time,end_time)",
            "futures_data = future_history.GetAllData()",
            "futures_data.reset_index(inplace = True)",
            "keep_columns(futures_data,['expiry','symbol','time','close','volume'])",
            "dct = df_to_dict(futures_data)",
            "print(dct)" ]
    
    df = execute_cell_code(cell_lines)
    
    return df

        
def get_option_bars(sym,start_time,end_time,strike_date,spread_type,strikes):
    
#    if sym == 'HES':
#        sym = 'AHC'

    min_strike = min(strikes)
    max_strike = max(strikes)
    if (max_strike - min_strike)/ min_strike > .8:
        print("****************************************    error in strikes")
        return

    for strike in strikes:
        if len(mtd.search_options_error(c_o_e,'MissingStrike',sym,start_time,end_time,strike_date,spread_type,strike)):
            print("Pre existing error MissingStrike",sym,strike_date,spread_type,strike)
            continue
        if len(mtd.search_options_error(c_o_e,'MissingExpire',sym,start_time,end_time,strike_date,spread_type,strike)):
            print("Pre existing error MissingExpire",sym,strike_date,spread_type,strike)
            continue
       
        
        df = mtd.search_options_cache(c_o,sym,strike_date,strike,spread_type,start_time,end_time)
        
        if len(df) == 0:
            print("downloading option",sym,spread_type,strike)
            
            try:
                df = get_option(sym,start_time,end_time,strike_date,spread_type,strike)
                if len(df):
                    df['expiry'] = [pd.to_datetime(ts) for ts in df['expiry']]

                
            except MissingStrikeError as e:
                info = "{} {}".format(e.lt_strike,e.gt_strike)
                mtd.add_options_error(c_o_e,"MissingStrike",sym,start_time,end_time,strike_date,spread_type,strike,info)
                print("MissingStrikeError",e.eq_strike,e.lt_strike,e.gt_strike)
                df = pd.DataFrame()
            except MissingExpireError as e:
                info = "{} {}".format(e.lt_expire,e.gt_expire)
                mtd.add_options_error(c_o_e,"MissingExpire",sym,start_time,end_time,strike_date,spread_type,strike,info)
                print("missingexpireerror",e.eq_expire,e.lt_expire,e.gt_expire)
                df = pd.DataFrame()
            
            print("get_option length:",len(df))
                    
                
            if len(df):
                df['symbol'] = sym  #symbol return is not always symbol searched for
                mtd.add_options_cache(c_o,sym,df)
        
def get_underlying_bars(collection,sym,start_time,end_time,keep_list = []):
        
    df = mtd.search_underlying_cache(collection,sym,start_time,end_time)
        
    if len(df) == 0:
        print("downloading underlying",sym)
        df = get_history(sym,start_time,end_time,keep_list)
        if len(df):
            df['symbol'] = sym  #symbol return is not always symbol searched for
            #df['symbol'] = [ sym.split()[0] for sym in df['symbol'] ]        
            mtd.add_underlying_cache(collection,sym,df)
        else:
            print("could not download:",sym)
    else:
        print("found in underlying_cache",sym)
        
def get_futures_bars(collection,sym,start_time,end_time,keep_list = []):
        
    df = mtd.search_futures_cache(collection,sym,start_time,end_time)
        
    if len(df) == 0:
        print("downloading underlying",sym)
        df = get_futures(sym,start_time,end_time,keep_list)
        if len(df):
            df['symbol'] = sym  #symbol return is not always symbol searched for
            #df['symbol'] = [ sym.split()[0] for sym in df['symbol'] ]        
            mtd.add_futures_cache(collection,sym,df)
        else:
            print("could not download:",sym)
    else:
        print("found in futures cache",sym)




## load project and notebook

In [ ]:
QC_INFO_PATH = '/home/cc/QuantConnect/qc_info.json'

with open(QC_INFO_PATH,'r') as infile:
    qc_info = json.load(infile)

driver = webdriver.Chrome("/usr/local/bin/chromedriver")

driver.get(qc_info['url_main'])

manual_login(driver,qc_info)




In [ ]:
load_project(driver,qc_info)

load_research_notebook(driver)

#tm.sleep(2)



In [ ]:
cell = get_cell(0)
enter_cell(cell)
execute_cell(cell)


# Load data

In [ ]:
import json
import datetime as dt
from datetime import timedelta
import pystore


import mt_pystore as mtd

pystore.set_path("pystore_mt")
store_mt = pystore.store('mt_store')
c_o = store_mt.collection('mt_options')
c_o_e = store_mt.collection('mt_options_error')
c_u = store_mt.collection('mt_underlying')


actions = json.loads(mtd.download("https://www.dropbox.com/s/psgut932tw77b82/actions.json?dl=1"))
print(actions[0])

ignore_sym_list = {
    'HES': "Can't find correct options. HES changes to AHC",
    'SPX': 'QC does not have data'}



# Process Events

In [ ]:


HOURS_TO_EST = 3

#EVENT_LIMIT = 125

def process_events():
    

    for i,event in enumerate(actions):
            
#        if i > EVENT_LIMIT:
#            break
    
    
        if all(item in event.keys() for item in  ['action','sym','spread_type','strike_date','strikes']):
        
            if event['action'] != 'shorting':
                continue
                
                
            print(i,event['sym'],event['strike_date'],event['strikes'])
            
#            if event['sym'] != "FSLY":
#                continue
            
            ts = dt.datetime.strptime(event['ts'], '%Y-%m-%dT%H:%M:%S') + timedelta(hours=HOURS_TO_EST)
        
            if event['sym'] in ignore_sym_list.keys():
                print(event['sym'],ignore_sym_list[event['sym']])
                continue        
        
            end_date = dt.datetime.strptime(event['strike_date'],'%Y-%m-%d')
            end_time = end_date.replace(hour = 16,minute = 0)
            
#            temp_end_ts = ts + timedelta(minutes = 2)
        
            get_underlying_bars(c_u,event['sym'],ts,end_time,['symbol','time','close'])

        
            get_option_bars(event['sym'],ts,end_time,end_time,event['spread_type'],event['strikes'])

        
        
            
        
            
            


In [ ]:
keep_trying = True

while keep_trying:
    try:
        cell = get_cell(0)
        enter_cell(cell)
        execute_cell(cell)
        tm.sleep(2)
        process_events()
        keep_trying = False
    except QcResponseError as e:
        print("except 4")
        print("QcResponse error:",e.error_string)
        wait_minutes(5)

        

print("finished")



### symbols that convert after get_history

HES to AHC

## symbol download

In [ ]:
import pystore
import mt_pystore as mtd


pystore.set_path("pystore_large")
store_large = pystore.store('store_large')
c_u = store_large.collection('underlying')


In [ ]:
from pandas_market_calendars.exchange_calendar_nyse import NYSEExchangeCalendar
nysecal = NYSEExchangeCalendar()
nysesch = nysecal.schedule('2019-01-01', '2020-01-01')

all_dates = [date for date in nysesch.index]

dt.datetime(year = 2020,month = 7,day = 3) in nysesch.index


In [ ]:
print(len(all_dates))

In [ ]:
num_days = 20
for x1 in range(0,len(all_dates),num_days):
    x2 = min(x1 + num_days,len(all_dates))-1
    
    start_time = all_dates[x1].replace(hour = 9,minute = 31)
    end_time = all_dates[x2].replace(hour = 16,minute = 0)
    
    print(x1,start_time)
    
    get_underlying_bars(c_u,'SPY',start_time,end_time,['symbol','time','close','volume'])

In [ ]:
import datetime as dt
start_time = dt.datetime(2018,1,1,9,31)
end_time = dt.datetime(2019,1,1,9,30)
keep_list = ['symbol','time','close','volume']
    
get_underlying_bars(c_u,'SPY',start_time,end_time,keep_list)

## Add to large from mt

In [ ]:
import pystore
import random

pystore.set_path("pystore_mt")
print("stores",pystore.list_stores())
mt_store = pystore.store('mt_store')
c_u = mt_store.collection('mt_underlying')
syms = list(c_u.list_items())

random.shuffle(syms)

print(syms)


pystore.set_path("pystore_large")
store_large = pystore.store('store_large')
c_u_large = store_large.collection('underlying')




In [ ]:
start_time = dt.datetime(2021,1,1,9,31)
end_time = dt.datetime(2021,7,5,14,30)
keep_list = ['symbol','time','close','volume']


for i,item in enumerate(syms):
    print(i,item)
    get_underlying_bars(c_u_large,item,start_time,end_time,['symbol','time','close','volume'])
    
print("finished")
    
    

## Add futures to large

In [ ]:
import pystore
import random

pystore.set_path("pystore_large")
store_large = pystore.store('store_large')
c_f_large = store_large.collection('futures')


In [ ]:
start_time = dt.datetime(2015,1,1,9,31)
end_time = dt.datetime(2021,7,10,9,31)
keep_list = ['expiry,''symbol','time','close','volume']

date_index = pd.date_range(start_time,end_time,freq = 'W')
date_list = [date for date in date_index]
if date_list[-1] != end_time:
    date_list.append(end_time)

for x in range(len(date_list)-1):
    start_time = date_list[x]
    end_time = date_list[x+1]
    get_futures_bars(c_f_large,'ES',start_time,end_time,keep_list)
    
print("finished")


## Some checks on large

In [ ]:
pystore.set_path("pystore_large")
store_large = pystore.store('store_large')
c_u_large = store_large.collection('underlying')

for i,sym in enumerate(c_u_large.list_items()):
    
    item = c_u_large.item(sym)
    
    data = item.data  # <-- Dask dataframe (see dask.pydata.org)
    metadata = item.metadata
    underlying_cache = item.to_pandas()
    print(i,sym,len(underlying_cache))




In [ ]:
6.5 * 4

In [ ]:
start_time = dt.datetime(2020,6,5,9,31)

[start_time + timedelta(minutes = 15) *x for x in range(27)]

In [ ]:
date_index = pd.date_range(start="2017-05-05",end = "2019-05-05",freq = 'W')

In [ ]:
for date in date_index:
    print(date)

In [ ]:
from math import nan

In [ ]:
rstr = [{'expiry': ('2017-03-17 13:30:00', [{'symbol': ('ES WIXFAL95L5OH', [{'time': ('2016-12-25 18:01:00', {'close': [2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625, 2258.625], 'volume': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]})}, {'time': ('2016-12-27 00:01:00', {'close': [2258.5, 2259.0, 2259.0, 2259.0, 2259.0, 2259.25, 2259.0, 2259.0, 2259.0, 2259.0, 2259.0, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.5, 2259.5, 2259.25, 2259.5, 2259.75, 2259.75, 2259.75, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.75, 2259.75, 2259.75, 2259.75, 2259.75, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.75, 2259.75, 2259.75, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.75, 2259.75, 2259.75, 2259.75, 2259.75, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.75, 2259.75, 2259.5, 2259.5, 2259.5, 2259.75, 2259.75, 2259.75, 2259.75, 2259.75, 2259.75, 2259.75, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.75, 2259.75, 2259.75, 2259.75, 2259.75, 2259.75, 2259.75, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.25, 2259.25, 2259.25, 2259.25, 2259.5, 2259.5, 2259.25, 2259.25, 2259.25, 2259.25, 2259.0, 2259.0, 2259.0, 2259.5, 2259.5, 2259.25, 2259.25, 2259.5, 2259.5, 2259.5, 2259.5, 2259.5, 2259.25, 2259.25, 2259.25, 2259.25, 2259.0, 2259.0, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.5, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.25, 2259.5, 2259.25, 2259.25, 2259.25, 2259.5, 2259.25, 2259.5, 2259.5, 2259.5, 2260.0, 2260.0, 2260.0, 2259.5, 2259.75, 2259.75, 2259.75, 2259.25, 2259.5, 2259.25, 2259.0, 2259.25, 2259.25, 2259.25, 2259.25, 2259.0, 2259.0, 2259.25, 2259.5, 2259.25, 2259.25, 2259.25, 2259.5, 2259.25, 2259.25, 2259.25, 2259.25, 2259.0, 2259.0, 2258.75, 2258.75, 2259.0, 2259.0, 2258.75, 2258.75, 2258.75, 2259.0, 2259.0, 2258.75, 2258.75, 2258.75, 2259.0, 2259.0, 2259.0, 2258.75, 2258.75, 2258.75, 2259.0, 2259.0, 2259.0, 2258.75, 2259.0, 2259.25, 2259.0, 2259.0, 2258.75, 2259.0, 2259.0, 2259.25, 2259.25, 2259.25, 2259.0, 2259.0, 2259.25, 2259.0, 2259.0, 2259.0, 2258.75, 2259.0, 2259.0, 2259.0, 2259.0, 2259.0, 2259.0, 2259.0, 2259.0, 2259.0, 2259.0, 2259.0, 2258.75, 2259.0, 2259.0, 2258.75, 2258.75, 2258.5, 2258.75, 2258.5, 2258.5, 2258.5, 2258.5, 2258.5, 2258.75, 2258.75, 2258.5, 2258.25, 2258.25, 2258.25, 2258.25, 2258.5, 2258.5, 2258.5, 2258.75, 2258.75, 2258.75, 2258.75, 2259.0, 2259.0, 2259.0, 2259.0, 2259.0, 2258.75, 2259.0, 2258.75, 2258.75, 2258.75, 2259.0, 2258.75, 2258.75, 2258.75, 2258.75, 2259.0, 2258.75, 2258.75, 2258.75, 2258.5, 2258.5, 2258.5, 2258.5, 2258.5, 2258.75, 2258.75, 2258.75, 2258.5, 2258.75, 2258.5, 2258.5, 2258.75, 2258.75, 2258.5, 2258.75, 2258.5, 2258.5, 2258.5, 2258.5, 2258.5, 2258.75, 2258.5, 2258.5, 2258.75, 2259.0, 2259.0, 2258.75, 2259.0, 2258.75, 2258.5, 2258.5, 2258.5, 2258.5, 2258.5, 2258.5, 2258.75, 2258.75, 2258.75, 2258.75, 2259.0, 2258.5, 2258.75, 2258.5, 2258.75, 2258.5, 2258.5, 2258.75, 2258.5, 2258.5, 2258.25, 2258.25, 2258.5, 2258.5, 2258.5, 2258.5, 2258.25, 2258.0, 2258.25, 2258.25, 2258.5, 2258.5, 2258.5, 2258.5, 2258.25, 2259.0, 2259.25, 2259.0, 2259.5, 2259.25, 2259.25, 2259.25, 2259.5, 2259.25, 2259.5, 2259.5, 2259.5, 2259.75, 2259.75, 2259.75, 2260.0, 2260.5, 2260.5, 2260.5, 2260.75, 2261.0, 2261.0, 2261.25, 2261.25, 2261.5, 2261.0, 2261.0, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2261.5, 2261.25, 2261.25, 2261.5, 2261.0, 2261.0, 2261.25, 2261.25, 2261.25, 2261.5, 2261.25, 2261.25, 2261.5, 2261.5, 2261.25, 2261.25, 2261.5, 2261.5, 2261.25, 2261.25, 2261.0, 2261.0, 2261.0, 2261.25, 2261.0, 2261.0, 2261.0, 2261.0, 2261.0, 2260.75, 2261.25, 2261.25, 2261.0, 2261.0, 2261.0, 2261.0, 2261.75, 2261.5, 2261.75, 2261.5, 2261.5, 2261.5, 2261.5, 2261.5, 2261.25, 2261.25, 2261.25, 2261.25, 2261.0, 2261.0, 2261.0, 2261.25, 2261.25, 2261.0, 2261.0, 2260.75, 2260.75, 2261.0, 2261.25, 2261.25, 2261.0, 2261.25, 2261.25, 2261.25, 2261.25, 2261.0, 2261.0, 2260.75, 2260.75, 2260.75, 2261.0, 2260.75, 2260.75, 2261.0, 2260.75, 2261.0, 2261.0, 2261.25, 2261.25, 2261.25, 2261.5, 2261.0, 2261.0, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2260.75, 2260.5, 2260.75, 2260.5, 2260.5, 2260.5, 2260.5, 2260.75, 2260.5, 2260.5, 2260.75, 2261.0, 2261.5, 2261.25, 2261.0, 2261.25, 2261.0, 2261.0, 2261.0, 2261.25, 2261.5, 2261.5, 2261.25, 2261.5, 2261.5, 2261.25, 2261.25, 2261.5, 2261.5, 2261.5, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2261.5, 2261.5, 2261.5, 2261.25, 2261.25, 2261.75, 2262.0, 2262.25, 2262.25, 2262.5, 2262.5, 2262.5, 2262.25, 2262.25, 2262.25, 2262.25, 2262.0, 2262.0, 2262.25, 2262.25, 2262.0, 2262.0, 2262.25, 2262.25, 2262.25, 2262.25, 2262.25, 2262.0, 2262.0, 2262.25, 2262.25, 2262.25, 2262.25, 2262.25, 2262.25, 2262.0, 2262.0, 2262.25, 2262.25, 2262.0, 2261.75, 2262.0, 2262.25, 2262.5, 2262.0, 2262.5, 2262.75, 2262.75, 2263.75, 2264.25, 2265.25, 2265.0, 2265.0, 2265.25, 2265.25, 2266.0, 2265.5, 2265.5, 2265.5, 2265.25, 2266.5, 2267.75, 2268.5, 2268.75, 2268.25, 2268.75, 2268.75, 2268.75, 2268.25, 2268.75, 2268.5, 2268.75, 2268.5, 2268.5, 2268.25, 2268.25, 2268.5, 2268.0, 2268.0, 2267.75, 2268.0, 2268.0, 2268.25, 2269.25, 2269.0, 2268.5, 2268.5, 2268.25, 2268.25, 2268.0, 2268.0, 2268.25, 2268.0, 2268.0, 2268.0, 2268.25, 2268.0, 2267.75, 2268.0, 2268.0, 2268.0, 2268.25, 2268.0, 2268.0, 2268.0, 2268.0, 2268.25, 2268.25, 2268.0, 2268.0, 2268.25, 2268.0, 2267.75, 2268.0, 2268.0, 2268.0, 2268.0, 2268.0, 2268.25, 2268.0, 2267.75, 2268.0, 2267.75, 2268.25, 2267.75, 2268.25, 2268.0, 2267.75, 2267.75, 2267.75, 2267.75, 2267.5, 2267.5, 2267.75, 2267.75, 2268.0, 2268.0, 2267.75, 2268.0, 2268.25, 2267.25, 2267.25, 2267.25, 2267.75, 2267.25, 2267.25, 2267.5, 2266.75, 2266.75, 2266.5, 2266.75, 2266.75, 2266.25, 2266.5, 2266.25, 2266.25, 2266.25, 2266.0, 2266.0, 2266.0, 2266.25, 2266.0, 2266.0, 2266.25, 2266.5, 2266.75, 2267.0, 2266.75, 2267.0, 2266.75, 2267.0, 2267.0, 2266.75, 2267.0, 2266.75, 2266.75, 2266.75, 2266.5, 2266.5, 2266.5, 2266.25, 2266.0, 2265.75, 2266.0, 2266.0, 2266.0, 2265.75, 2265.5, 2265.25, 2265.25, 2265.25, 2265.25, 2265.5, 2265.75, 2265.5, 2265.25, 2265.25, 2265.25, 2265.0, 2265.0, 2264.75, 2264.75, 2265.0, 2265.0, 2265.25, 2265.25, 2265.25, 2265.0, 2265.0, 2264.5, 2264.75, 2264.75, 2264.5, 2265.0, 2265.0, 2265.0, 2265.0, 2264.75, 2264.75, 2264.75, 2264.75, 2265.0, 2264.75, 2265.25, 2265.0, 2265.0, 2265.0, 2264.75, 2265.0, 2265.0, 2264.75, 2264.5, 2264.5, 2264.5, 2264.5, 2265.0, 2265.0, 2265.25, 2265.5, 2265.0, 2265.25, 2265.0, 2265.0, 2265.5, 2265.25, 2265.5, 2265.5, 2265.25, 2265.25, 2265.25, 2265.25, 2265.25, 2265.25, 2265.25, 2265.5, 2265.0, 2265.25, 2265.25, 2265.5, 2265.5, 2265.0, 2265.0, 2265.25, 2265.5, 2265.5, 2265.5, 2265.75, 2265.75, 2266.0, 2265.75, 2265.75, 2265.75, 2265.5, 2265.25, 2265.5, 2265.5, 2265.75, 2265.5, 2265.5, 2265.75, 2265.5, 2266.0, 2265.75, 2266.0, 2265.75, 2266.0, 2266.0, 2265.75, 2266.0, 2266.25, 2266.25, 2266.25, 2266.0, 2266.25, 2266.75, 2266.5, 2266.25, 2266.0, 2266.0, 2266.25, 2266.5, 2266.5, 2266.0, 2266.0, 2266.0, 2266.25, 2266.25, 2266.0, 2266.25, 2265.5, 2265.5, 2265.25, 2265.25, 2265.75, 2266.0, 2265.75, 2265.25, 2265.0, 2265.0, 2265.25, 2265.0, 2265.25, 2265.5, 2265.5, 2265.75, 2266.0, 2265.5, 2265.5, 2266.0, 2266.0, 2265.75, 2266.75, 2266.75, 2266.5, 2266.75, 2266.5, 2266.75, 2266.75, 2266.75, 2266.5, 2266.5, 2266.5, 2266.75, 2266.5, 2266.5, 2266.25, 2266.25, 2266.0, 2266.0, 2266.0, 2266.0, 2266.0, 2266.0, 2266.25, 2266.25, 2266.25, 2266.25, 2266.0, 2266.0, 2266.5, 2266.25, 2266.5, 2266.5, 2266.5, 2266.75, 2266.25, 2266.0, 2265.5, 2266.0, 2265.75, 2266.0, 2266.0, 2265.5, 2265.75, 2265.5, 2265.75, 2265.5, 2265.75, 2266.0, 2266.5, 2266.0, 2266.25, 2265.75, 2266.0, 2266.0, 2265.5, 2265.75, 2265.25, 2265.5, 2265.25, 2265.5, 2265.5, 2265.25, 2265.0, 2265.0, 2265.25, 2265.0, 2265.0, 2265.5, 2265.5, 2265.25, 2265.25, 2265.25, 2265.5, 2265.5, 2265.0, 2265.0, 2264.75, 2264.75, 2265.0, 2264.75, 2264.75, 2264.75, 2264.75, 2264.5, 2264.5, 2265.25, 2265.25, 2265.25, 2265.0, 2265.0, 2265.25, 2264.25, 2263.75, 2264.0, 2263.75, 2264.0, 2264.25, 2264.5, 2264.0, 2264.0, 2264.0, 2263.75, 2263.5, 2264.0, 2263.25, 2262.5, 2262.75, 2262.25, 2261.75, 2262.0, 2261.75, 2261.5, 2261.75, 2261.5, 2261.5, 2261.5, 2261.0, 2260.75, 2260.75], 'volume': [9.0, 55.0, 11.0, 9.0, 9.0, 31.0, 52.0, 10.0, 10.0, 12.0, 11.0, 12.0, 14.0, 14.0, 10.0, 9.0, 55.0, 18.0, 10.0, 9.0, 13.0, 12.0, 9.0, 10.0, 60.0, 11.0, 9.0, 12.0, 7.0, 46.0, 11.0, 38.0, 42.0, 11.0, 13.0, 12.0, 60.0, 7.0, 16.0, 12.0, 11.0, 10.0, 9.0, 28.0, 21.0, 9.0, 9.0, 13.0, 9.0, 11.0, 56.0, 10.0, 14.0, 9.0, 20.0, 14.0, 32.0, 49.0, 9.0, 25.0, 280.0, 57.0, 12.0, 186.0, 30.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 237.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 4.0, 33.0, 2.0, 17.0, 0.0, 6.0, 1.0, 0.0, 0.0, 0.0, 9.0, 3.0, 3.0, 0.0, 13.0, 28.0, 0.0, 1.0, 0.0, 1.0, 0.0, 37.0, 52.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 13.0, 0.0, 0.0, 20.0, 0.0, 7.0, 0.0, 0.0, 3.0, 82.0, 46.0, 34.0, 121.0, 59.0, 27.0, 10.0, 7.0, 9.0, 42.0, 16.0, 20.0, 91.0, 2.0, 33.0, 12.0, 2.0, 1.0, 1.0, 3.0, 2.0, 0.0, 1.0, 23.0, 29.0, 0.0, 8.0, 3.0, 8.0, 12.0, 2.0, 2.0, 3.0, 0.0, 33.0, 8.0, 0.0, 65.0, 1.0, 25.0, 0.0, 2.0, 5.0, 28.0, 1.0, 32.0, 0.0, 6.0, 49.0, 18.0, 44.0, 0.0, 145.0, 0.0, 93.0, 92.0, 1.0, 10.0, 3.0, 55.0, 289.0, 168.0, 87.0, 71.0, 18.0, 12.0, 21.0, 31.0, 2.0, 69.0, 34.0, 6.0, 2.0, 10.0, 19.0, 220.0, 9.0, 12.0, 15.0, 69.0, 61.0, 27.0, 9.0, 3.0, 103.0, 169.0, 4.0, 4.0, 2.0, 69.0, 247.0, 31.0, 12.0, 92.0, 7.0, 5.0, 75.0, 44.0, 18.0, 37.0, 10.0, 223.0, 10.0, 95.0, 23.0, 18.0, 33.0, 134.0, 19.0, 74.0, 20.0, 10.0, 5.0, 9.0, 8.0, 3.0, 4.0, 29.0, 34.0, 7.0, 176.0, 145.0, 110.0, 12.0, 2.0, 19.0, 30.0, 24.0, 19.0, 1.0, 1.0, 2.0, 9.0, 2.0, 7.0, 86.0, 10.0, 72.0, 115.0, 10.0, 68.0, 13.0, 0.0, 11.0, 2.0, 48.0, 137.0, 114.0, 2.0, 9.0, 90.0, 5.0, 1.0, 9.0, 3.0, 37.0, 25.0, 14.0, 0.0, 3.0, 1.0, 3.0, 13.0, 5.0, 32.0, 6.0, 7.0, 5.0, 4.0, 1.0, 3.0, 8.0, 4.0, 6.0, 4.0, 2.0, 53.0, 0.0, 2.0, 9.0, 3.0, 20.0, 0.0, 16.0, 7.0, 13.0, 4.0, 137.0, 6.0, 0.0, 1.0, 28.0, 4.0, 1.0, 0.0, 2.0, 6.0, 57.0, 39.0, 8.0, 84.0, 9.0, 18.0, 3.0, 6.0, 117.0, 12.0, 0.0, 11.0, 88.0, 2.0, 23.0, 66.0, 24.0, 0.0, 0.0, 8.0, 15.0, 1.0, 21.0, 4.0, 2.0, 7.0, 15.0, 18.0, 1.0, 689.0, 2.0, 61.0, 0.0, 50.0, 19.0, 97.0, 87.0, 14.0, 13.0, 27.0, 9.0, 6.0, 4.0, 43.0, 568.0, 148.0, 68.0, 159.0, 163.0, 157.0, 12.0, 469.0, 152.0, 48.0, 4.0, 42.0, 8.0, 11.0, 8.0, 372.0, 303.0, 314.0, 207.0, 37.0, 385.0, 65.0, 725.0, 50.0, 106.0, 32.0, 37.0, 61.0, 63.0, 29.0, 60.0, 31.0, 20.0, 18.0, 14.0, 7.0, 21.0, 48.0, 117.0, 37.0, 11.0, 4.0, 83.0, 43.0, 69.0, 25.0, 86.0, 27.0, 13.0, 6.0, 5.0, 7.0, 12.0, 13.0, 168.0, 9.0, 9.0, 8.0, 122.0, 18.0, 79.0, 9.0, 31.0, 6.0, 8.0, 9.0, 22.0, 54.0, 45.0, 26.0, 9.0, 1.0, 5.0, 0.0, 229.0, 27.0, 20.0, 19.0, 15.0, 38.0, 5.0, 45.0, 28.0, 12.0, 66.0, 2.0, 1.0, 2.0, 14.0, 7.0, 11.0, 4.0, 27.0, 72.0, 11.0, 4.0, 115.0, 265.0, 20.0, 15.0, 38.0, 12.0, 2.0, 3.0, 27.0, 77.0, 37.0, 16.0, 13.0, 9.0, 2.0, 22.0, 33.0, 16.0, 11.0, 228.0, 37.0, 11.0, 130.0, 117.0, 40.0, 370.0, 5.0, 215.0, 69.0, 60.0, 142.0, 172.0, 162.0, 141.0, 8.0, 11.0, 13.0, 99.0, 152.0, 12.0, 3.0, 5.0, 429.0, 19.0, 285.0, 257.0, 116.0, 30.0, 127.0, 3.0, 55.0, 26.0, 379.0, 165.0, 25.0, 23.0, 57.0, 25.0, 13.0, 18.0, 14.0, 14.0, 69.0, 19.0, 20.0, 2.0, 31.0, 73.0, 53.0, 4.0, 16.0, 45.0, 150.0, 154.0, 239.0, 591.0, 241.0, 196.0, 47.0, 140.0, 230.0, 61.0, 449.0, 43.0, 55.0, 28.0, 39.0, 315.0, 235.0, 15.0, 49.0, 18.0, 3.0, 122.0, 375.0, 62.0, 27.0, 47.0, 49.0, 25.0, 17.0, 177.0, 103.0, 40.0, 79.0, 134.0, 453.0, 139.0, 409.0, 123.0, 242.0, 228.0, 1792.0, 6708.0, 1964.0, 1954.0, 3235.0, 4116.0, 5684.0, 2842.0, 2517.0, 1074.0, 2739.0, 4460.0, 2610.0, 1500.0, 1351.0, 1145.0, 6606.0, 8477.0, 3850.0, 8533.0, 4681.0, 3498.0, 1059.0, 4524.0, 4093.0, 3174.0, 3216.0, 1137.0, 1206.0, 504.0, 956.0, 2079.0, 1622.0, 1578.0, 1910.0, 1827.0, 579.0, 142.0, 2940.0, 6406.0, 1662.0, 1388.0, 1148.0, 1107.0, 1027.0, 568.0, 805.0, 323.0, 418.0, 842.0, 379.0, 446.0, 1628.0, 310.0, 732.0, 176.0, 1681.0, 1160.0, 586.0, 955.0, 212.0, 1496.0, 71.0, 773.0, 825.0, 795.0, 809.0, 775.0, 318.0, 1142.0, 886.0, 543.0, 796.0, 510.0, 824.0, 1153.0, 104.0, 378.0, 148.0, 499.0, 2168.0, 1573.0, 1194.0, 588.0, 10.0, 165.0, 123.0, 346.0, 623.0, 457.0, 325.0, 342.0, 829.0, 1123.0, 1008.0, 859.0, 1970.0, 1163.0, 1703.0, 747.0, 652.0, 55.0, 758.0, 665.0, 1550.0, 358.0, 1469.0, 863.0, 979.0, 750.0, 460.0, 654.0, 875.0, 123.0, 182.0, 308.0, 1921.0, 907.0, 712.0, 346.0, 1218.0, 366.0, 735.0, 2176.0, 342.0, 216.0, 273.0, 67.0, 241.0, 743.0, 513.0, 141.0, 83.0, 94.0, 1046.0, 160.0, 198.0, 837.0, 795.0, 1302.0, 730.0, 101.0, 417.0, 517.0, 1179.0, 697.0, 2826.0, 346.0, 1506.0, 498.0, 816.0, 206.0, 1206.0, 91.0, 1089.0, 587.0, 1144.0, 801.0, 1300.0, 329.0, 395.0, 94.0, 473.0, 56.0, 2130.0, 1716.0, 885.0, 423.0, 672.0, 932.0, 1132.0, 368.0, 435.0, 268.0, 801.0, 71.0, 539.0, 154.0, 26.0, 389.0, 636.0, 55.0, 517.0, 38.0, 90.0, 144.0, 96.0, 344.0, 156.0, 226.0, 158.0, 528.0, 155.0, 699.0, 642.0, 580.0, 29.0, 12.0, 43.0, 642.0, 219.0, 219.0, 72.0, 70.0, 25.0, 144.0, 74.0, 88.0, 71.0, 1020.0, 644.0, 530.0, 467.0, 432.0, 132.0, 198.0, 1374.0, 132.0, 907.0, 45.0, 287.0, 559.0, 208.0, 485.0, 89.0, 1054.0, 226.0, 311.0, 1861.0, 332.0, 1229.0, 396.0, 570.0, 54.0, 64.0, 89.0, 1142.0, 1145.0, 80.0, 433.0, 36.0, 151.0, 71.0, 139.0, 83.0, 904.0, 102.0, 562.0, 41.0, 112.0, 973.0, 765.0, 34.0, 612.0, 1276.0, 753.0, 69.0, 60.0, 1096.0, 456.0, 66.0, 45.0, 72.0, 210.0, 71.0, 929.0, 255.0, 682.0, 239.0, 1321.0, 35.0, 1049.0, 679.0, 337.0, 86.0, 1204.0, 1022.0, 1819.0, 1747.0, 67.0, 494.0, 587.0, 430.0, 1342.0, 992.0, 36.0, 421.0, 1554.0, 1113.0, 107.0, 39.0, 525.0, 62.0, 668.0, 527.0, 1244.0, 113.0, 596.0, 1498.0, 1490.0, 782.0, 153.0, 1341.0, 874.0, 809.0, 515.0, 360.0, 24.0, 73.0, 974.0, 169.0, 492.0, 621.0, 16.0, 18.0, 638.0, 1121.0, 822.0, 793.0, 1856.0, 29.0, 1544.0, 805.0, 1230.0, 543.0, 1207.0, 35.0, 359.0, 541.0, 905.0, 1302.0, 625.0, 808.0, 753.0, 80.0, 1330.0, 1720.0, 1346.0, 2025.0, 924.0, 517.0, 1072.0, 677.0, 665.0, 1224.0, 1603.0, 1113.0, 761.0, 1162.0, 846.0, 1114.0, 1527.0, 1037.0, 305.0, 1348.0, 277.0, 121.0, 73.0, 199.0, 1075.0, 855.0, 1078.0, 897.0, 772.0, 48.0, 32.0, 180.0, 261.0, 132.0, 642.0, 1559.0, 1747.0, 1718.0, 1149.0, 675.0, 191.0, 1324.0, 199.0, 1843.0, 6012.0, 3196.0, 2790.0, 998.0, 2258.0, 1595.0, 903.0, 2765.0, 755.0, 4477.0, 3270.0, 24969.0, 21247.0, 8710.0, 1627.0, 3361.0, 2641.0, 1584.0, 1675.0, 966.0, 379.0, 1594.0, 657.0, 750.0, 835.0, 2163.0, 4790.0]})}, {'time': ('2016-12-27 16:31:00', {'close': [2261.5, 2261.5, 2261.75, 2261.75, 2261.75, 2261.5, 2261.5, 2261.5, 2261.75, 2261.5, 2261.5, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2261.25, 2260.75, 2260.75, 2261.25, 2261.0, 2261.0, 2261.0, 2260.75, 2261.0, 2261.0, 2261.0, 2260.75, 2260.5], 'volume': [987.0, 411.0, 513.0, 27.0, 69.0, 150.0, 91.0, 8.0, 9.0, 79.0, 30.0, 66.0, 68.0, 32.0, 80.0, 23.0, 185.0, 122.0, 189.0, 112.0, 102.0, 57.0, 54.0, 30.0, 75.0, 43.0, 132.0, 9.0, 217.0, 356.0]})}, {'time': ('2016-12-27 18:01:00', {'close': [2261.25, 2261.5, 2261.75, 2261.75, 2261.5, 2261.75, 2261.75, 2261.5, 2261.75, 2261.5, 2261.75, 2261.75, 2261.5, 2261.5, 2261.75, 2262.0, 2262.0, 2261.75, 2261.75, 2261.75, 2262.0, 2262.0, 2262.0, 2261.75, 2261.75, 2261.75, 2261.5, 2261.25, 2261.25, 2261.5, 2261.75, 2261.75, 2262.0, 2262.0, 2261.75, 2261.75, 2261.75, 2262.0, 2261.75, 2262.0, 2261.75, 2261.75, 2261.75, 2261.75, 2261.75, 2261.75, 2261.75, 2261.75, 2261.75, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2261.75, 2261.75, 2261.75, 2262.0, 2262.0, 2261.75, 2261.75, 2261.5, 2261.75, 2261.75, 2261.75, 2261.5, 2261.75, 2261.75, 2261.75, 2261.5, 2261.75, 2261.75, 2261.75, 2261.5, 2261.75, 2261.75, 2261.75, 2262.0, 2261.75, 2262.0, 2262.0, 2262.0, 2262.0, 2261.75, 2262.0, 2262.0, 2261.75, 2261.75, 2261.75, 2261.75, 2261.75, 2261.75, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2261.75, 2261.75, 2261.75, 2262.0, 2262.0, 2262.0, 2262.0, 2261.75, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2261.75, 2262.0, 2261.75, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2261.75, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2262.0, 2262.25, 2262.25, 2262.5, 2262.5, 2262.25, 2262.5, 2262.5, 2262.75, 2262.75, 2262.75, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.25, 2263.0, 2263.0, 2263.25, 2263.25, 2263.5, 2263.0, 2263.25, 2263.25, 2263.5, 2263.25, 2263.25, 2263.0, 2263.25, 2263.5, 2263.25, 2263.5, 2263.25, 2263.25, 2263.5, 2263.25, 2263.5, 2263.5, 2263.5, 2263.5, 2263.25, 2263.25, 2263.0, 2263.5, 2263.5, 2263.5, 2263.5, 2263.25, 2263.25, 2263.25, 2263.0, 2263.25, 2263.0, 2263.25, 2263.25, 2263.25, 2263.25, 2263.25, 2263.5, 2263.5, 2263.5, 2263.25, 2263.5, 2263.25, 2263.5, 2263.25, 2263.25, 2263.0, 2263.25, 2262.75, 2263.0, 2262.75, 2262.75, 2262.75, 2263.0, 2263.0, 2262.75, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2262.75, 2262.75, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2262.75, 2262.75, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.25, 2263.0, 2263.25, 2263.25, 2263.0, 2263.0, 2263.25, 2263.25, 2263.25, 2263.0, 2263.25, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.25, 2263.25, 2263.25, 2263.25, 2263.25, 2263.0, 2263.0, 2263.25, 2263.25, 2263.25, 2263.5, 2263.5, 2263.25, 2263.5, 2263.5, 2263.25, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.25, 2262.75, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.25, 2263.25, 2263.0, 2263.25, 2263.5, 2263.5, 2263.5, 2263.0, 2263.0, 2263.0, 2263.0, 2263.25, 2263.25, 2263.0, 2263.0, 2263.25, 2263.25, 2263.25, 2263.25, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.25, 2263.25, 2263.25, 2263.25, 2263.25, 2263.0, 2263.25, 2263.25, 2263.0, 2263.25, 2263.0, 2263.25, 2263.25, 2263.0, 2263.25, 2263.25, 2263.0, 2263.25, 2263.0, 2263.0, 2263.25, 2263.25, 2263.0, 2263.0, 2263.0, 2263.0, 2262.75, 2263.0, 2262.75, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.0, 2263.25, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.0, 2263.25, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.25, 2263.5, 2263.5, 2263.5, 2263.5, 2263.25, 2263.25, 2263.25, 2263.5, 2263.5, 2263.5, 2263.5, 2263.25, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.5, 2263.25, 2263.5, 2263.5, 2263.5, 2263.5, 2263.75, 2263.75, 2263.75, 2263.75, 2263.75, 2263.75, 2263.75, 2263.75, 2263.75, 2263.75, 2263.75, 2263.75, 2264.0, 2263.75, 2264.0, 2264.0, 2264.0, 2263.75, 2264.0, 2264.0, 2264.5, 2264.5, 2264.75, 2264.75, 2264.75, 2264.5, 2264.5, 2264.5, 2264.5, 2264.5, 2264.75, 2264.75, 2264.5, 2264.5, 2264.5, 2264.5, 2264.5, 2264.5, 2264.5, 2264.5, 2264.5, 2264.5, 2264.75, 2264.75, 2264.75, 2264.5, 2264.5, 2264.5, 2264.75, 2264.75, 2264.75, 2265.0, 2265.0, 2265.0, 2264.75, 2265.0, 2265.0, 2265.0, 2265.0, 2264.75, 2265.25, 2265.25, 2265.25, 2265.0, 2265.0, 2264.75, 2265.0, 2265.0, 2265.0, 2265.0, 2265.0, 2265.25, 2265.5, 2265.0, 2265.5, 2265.25, 2265.25, 2265.0, 2265.25, 2265.25, 2265.5, 2265.25, 2265.5, 2265.5, 2265.5, 2265.75, 2265.5, 2265.75, 2266.0, 2266.0, 2266.0, 2266.0, 2266.0, 2266.0, 2266.0, 2266.0, 2266.5, 2266.25, 2265.75, 2266.0, 2266.0, 2265.75, 2265.75, 2265.75, 2265.75, 2265.75, 2265.75, 2265.75, 2265.75, 2265.75, 2265.75, 2265.5, 2265.75, 2265.75, 2265.75, 2265.75, 2265.5, 2265.75, 2265.5, 2265.5, 2265.0, 2265.5, 2265.5, 2265.5, 2266.0, 2266.0, 2266.0, 2266.25, 2266.0, 2265.5, 2266.25, 2265.75, 2266.0, 2266.25, 2265.75, 2266.0, 2266.0, 2265.5, 2265.75, 2265.75, 2265.5, 2265.5, 2265.75, 2265.75, 2265.75, 2266.0, 2265.75, 2265.5, 2265.5, 2265.5, 2265.5, 2265.5, 2265.0, 2264.5, 2265.0, 2265.25, 2265.25, 2265.0, 2264.5, 2265.25, 2265.5, 2265.0, 2265.25, 2265.25, 2265.25, 2265.25, 2265.5, 2265.25, 2265.25, 2265.25, 2265.25, 2265.5, 2265.25, 2265.25, 2265.5, 2265.5, 2265.5, 2265.5, 2265.0, 2264.75, 2265.5, 2265.5, 2265.25, 2265.5, 2265.5, 2265.25, 2265.0, 2265.0, 2265.0, 2265.25, 2265.25, 2265.0, 2265.25, 2264.75, 2264.75, 2264.75, 2265.0, 2264.75, 2265.25, 2265.25, 2265.25, 2265.5, 2265.25, 2265.0, 2265.0, 2265.25, 2265.5, 2265.5, 2265.5, 2265.5, 2265.5, 2265.5, 2265.75, 2265.5, 2265.75, 2265.75, 2265.75, 2265.25, 2265.5, 2265.5, 2265.5, 2265.5, 2265.5, 2266.0, 2266.0, 2266.5, 2266.0, 2266.0, 2265.75, 2265.75, 2265.75, 2266.0, 2265.75, 2265.75, 2265.5, 2265.5, 2265.5, 2265.5, 2265.75, 2265.75, 2266.0, 2266.0, 2266.0, 2266.0, 2266.25, 2266.0, 2266.0, 2265.75, 2266.0, 2266.0, 2266.0, 2266.0, 2266.25, 2266.5, 2266.25, 2266.5, 2266.25, 2266.0, 2266.25, 2266.0, 2266.0, 2266.0, 2265.75, 2265.75, 2265.75, 2266.0, 2266.0, 2266.25, 2266.0, 2266.0, 2266.25, 2266.0, 2266.0, 2265.75, 2266.0, 2266.0, 2266.0, 2266.0, 2266.0, 2266.25, 2266.0, 2265.75, 2265.5, 2265.5, 2265.5, 2266.0, 2266.25, 2266.25, 2265.75, 2265.75, 2266.0, 2266.0, 2266.0, 2266.25, 2266.0, 2266.25, 2266.25, 2266.0, 2266.25, 2266.25, 2266.0, 2266.25, 2265.75, 2265.75, 2265.75, 2265.5, 2265.75, 2265.75, 2265.5, 2265.75, 2266.0, 2265.75, 2265.75, 2266.0, 2265.5, 2265.75, 2265.75, 2265.75, 2265.75, 2265.75, 2265.5, 2265.5, 2265.25, 2265.5, 2265.75, 2265.25, 2265.25, 2265.5, 2265.5, 2265.5, 2265.75, 2265.5, 2265.5, 2265.75, 2265.75, 2265.5, 2265.75, 2265.75, 2265.75, 2266.0, 2266.0, 2265.75, 2265.5, 2265.75, 2265.75, 2265.75, 2265.75, 2265.75, 2265.75, 2265.75, 2266.0, 2265.75, 2265.5, 2266.0, 2265.75, 2266.0, 2265.75, 2265.75, 2265.75, 2266.0, 2266.25, 2266.0, 2266.25, 2266.25, 2266.25, 2266.0, 2266.0, 2266.25, 2266.0, 2265.75, 2265.5, 2265.5, 2265.75, 2265.75, 2265.5, 2265.5, 2265.5, 2265.5, 2265.5, 2265.5, 2265.75, 2265.5, 2265.5, 2265.25, 2265.5, 2265.75, 2265.75, 2265.75, 2265.75, 2265.75, 2266.0, 2265.5, 2265.75, 2265.5, 2265.5, 2265.75, 2266.0, 2266.0, 2266.0, 2266.0, 2266.0, 2266.0, 2265.75, 2266.0, 2266.0, 2265.75, 2266.0, 2266.0, 2265.75, 2265.5, 2265.5, 2265.75, 2266.0, 2266.25, 2266.0, 2266.0, 2266.0, 2266.0, 2266.0, 2266.25, 2266.25, 2266.25, 2266.5, 2266.75, 2266.75, 2267.0, 2266.5, 2266.5, 2266.25, 2266.5, 2266.25, 2266.25, 2266.25, 2266.25, 2266.25, 2266.25, 2266.5, 2266.25, 2266.25, 2266.25, 2267.0, 2267.0, 2266.75, 2266.75, 2266.5, 2266.25, 2266.5, 2266.25, 2266.5, 2266.5, 2266.25, 2266.25, 2266.5, 2266.75, 2266.5, 2266.5, 2266.75, 2266.25, 2266.25, 2266.25, 2266.0, 2266.5, 2266.5, 2266.25, 2266.5, 2266.5, 2266.25, 2266.0, 2266.0, 2266.0, 2265.5, 2265.75, 2266.25, 2266.25, 2266.5, 2266.75, 2266.5, 2266.75, 2266.75, 2267.5, 2266.75, 2267.0, 2266.75, 2266.75, 2267.0, 2267.0, 2267.0, 2267.0, 2266.75, 2266.75, 2266.75, 2266.5, 2266.75, 2266.5, 2267.25, 2267.25, 2267.5, 2267.5, 2267.25, 2267.25, 2267.0, 2266.75, 2266.5, 2266.5, 2266.75, 2266.5, 2266.5, 2266.75, 2267.0, 2266.25, 2266.0, 2266.25, 2266.5, 2266.25, 2265.75, 2265.5, 2265.0, 2264.5, 2264.75, 2262.75, 2262.0, 2261.75, 2260.75, 2261.5, 2261.0, 2260.5, 2260.75, 2260.5, 2260.5, 2260.5, 2259.5, 2259.0, 2257.5, 2257.75, 2257.75, 2257.25, 2257.25, 2256.25, 2256.5, 2255.5, 2255.75, 2255.75, 2255.25, 2256.75, 2257.0, 2256.75, 2256.75, 2256.75, 2256.0, 2254.75, 2255.75, 2256.0, 2256.25, 2255.75, 2255.75, 2256.75, 2256.5, 2255.5, 2255.75, 2255.25, 2255.25, 2256.0, 2256.0, 2255.75, 2256.0, 2255.5, 2256.25, 2256.25, 2256.25, 2256.5, 2257.0, 2256.0, 2255.75, 2256.25, 2256.5, 2256.25, 2256.25, 2255.75, 2256.0, 2256.25, 2256.25, 2256.25, 2255.75, 2255.25, 2255.5, 2255.75, 2255.5, 2255.5, 2255.75, 2255.25, 2255.25, 2255.0, 2254.5, 2255.0, 2254.5, 2255.0, 2255.25, 2255.25, 2255.5, 2255.5, 2255.25, 2255.25, 2255.5, 2256.0, 2256.0, 2255.75, 2255.75, 2255.25, 2255.5, 2256.0, 2256.5, 2256.25, 2255.75, 2256.0, 2256.0, 2255.75, 2256.0, 2255.5, 2255.75, 2256.0, 2257.0, 2257.25, 2257.5, 2257.25, 2257.25, 2257.25, 2257.75, 2257.75, 2258.0, 2257.75, 2257.75, 2257.75, 2257.75, 2257.75, 2257.75, 2257.25, 2257.5, 2257.5, 2257.5, 2257.75, 2257.5, 2257.25, 2257.25, 2257.25, 2257.25, 2257.5, 2257.5, 2257.25, 2257.25, 2257.0, 2257.0, 2257.0, 2256.75, 2256.25, 2255.5, 2255.75, 2255.5, 2255.0, 2255.0, 2253.25, 2253.5, 2252.75, 2252.5, 2252.5, 2252.25, 2251.75, 2253.0, 2252.25, 2252.0, 2251.5, 2252.0, 2251.5, 2250.25, 2250.0, 2249.75, 2249.5, 2250.0, 2250.0, 2249.75, 2249.25, 2249.5, 2249.5, 2249.25, 2249.25, 2249.25, 2250.25, 2250.25, 2250.25, 2250.5, 2249.75, 2249.5, 2249.5, 2249.0, 2248.75, 2249.0, 2249.0, 2248.75, 2248.5, 2248.75, 2249.25, 2248.5, 2248.25, 2247.25, 2247.0, 2246.75, 2246.5, 2246.75, 2246.75, 2247.25, 2247.5, 2248.0, 2248.25, 2248.25, 2248.5, 2248.25, 2248.25, 2248.25, 2248.0, 2248.5, 2248.0, 2248.0, 2247.75, 2248.5, 2248.5, 2248.25, 2247.5, 2247.75, 2248.5, 2248.25, 2248.25, 2248.5, 2248.75, 2248.25, 2249.0, 2248.75, 2248.5, 2248.25, 2248.0, 2248.25, 2248.0, 2248.5, 2248.75, 2248.5, 2249.0, 2249.0, 2249.5, 2249.0, 2248.5, 2248.5, 2248.75, 2248.5, 2248.5, 2248.5, 2248.5, 2248.5, 2248.25, 2248.5, 2248.5, 2248.5, 2248.75, 2248.5, 2248.25, 2248.25, 2247.75, 2247.5, 2247.5, 2247.5, 2247.25, 2247.25, 2248.0, 2248.0, 2247.75, 2247.25, 2246.5, 2247.0, 2247.5, 2247.25, 2247.25, 2246.75, 2247.5, 2248.0, 2248.25, 2248.0, 2247.75, 2247.75, 2247.25, 2247.75, 2247.75, 2248.0, 2247.75, 2247.0, 2246.25, 2246.25, 2246.25, 2245.75, 2245.5, 2245.25, 2244.75, 2245.5, 2245.75, 2245.25, 2245.0, 2244.75, 2245.0, 2245.5, 2245.75, 2245.75, 2245.5, 2245.75, 2246.75, 2246.5, 2246.75, 2246.5, 2246.25, 2246.5, 2246.75, 2246.75, 2246.25, 2246.5, 2246.5, 2246.0, 2245.75, 2245.5, 2245.75, 2246.0, 2246.25, 2246.0, 2245.75, 2245.25, 2245.25, 2244.75, 2244.25, 2244.75, 2245.5, 2246.25, 2246.0, 2246.25, 2246.0, 2246.5, 2246.0, 2246.25, 2246.25, 2247.25, 2247.25, 2247.75, 2247.25, 2247.25, 2247.0, 2246.25, 2246.5, 2245.75, 2246.0, 2246.25, 2246.5, 2246.75, 2246.75, 2246.0, 2246.0, 2245.75, 2246.5, 2246.25, 2246.25, 2246.25, 2246.75, 2246.75, 2246.5, 2246.75, 2246.0, 2246.0, 2245.25, 2245.0, 2245.5, 2246.25, 2246.0, 2245.75, 2246.0, 2245.25, 2245.25, 2245.0, 2244.75, 2243.75, 2244.75, 2243.75, 2244.25, 2244.75, 2245.25, 2245.0, 2245.25, 2244.75, 2244.75, 2244.75, 2244.5, 2244.25, 2245.25, 2245.25, 2245.25, 2245.5, 2244.25, 2245.25, 2244.75, 2245.75, 2246.0, 2246.5, 2246.0, 2245.5, 2245.75, 2246.25, 2245.75, 2245.5, 2245.25, 2245.25, 2244.75, 2244.75, 2245.5], 'volume': [493.0, 256.0, 137.0, 12.0, 34.0, 87.0, 120.0, 21.0, 25.0, 67.0, 30.0, 1.0, 83.0, 4.0, 138.0, 125.0, 3.0, 33.0, 60.0, 14.0, 69.0, 3.0, 0.0, 1.0, 90.0, 0.0, 95.0, 107.0, 0.0, 19.0, 40.0, 40.0, 11.0, 6.0, 19.0, 0.0, 40.0, 16.0, 8.0, 1.0, 3.0, 0.0, 1.0, 1.0, 1.0, 0.0, 2.0, 0.0, 2.0, 2.0, 4.0, 0.0, 24.0, 1.0, 179.0, 32.0, 28.0, 25.0, 39.0, 6.0, 132.0, 67.0, 11.0, 4.0, 5.0, 87.0, 11.0, 7.0, 13.0, 108.0, 35.0, 25.0, 11.0, 4.0, 4.0, 46.0, 2.0, 7.0, 4.0, 2.0, 14.0, 15.0, 0.0, 8.0, 5.0, 3.0, 47.0, 60.0, 33.0, 17.0, 25.0, 4.0, 10.0, 47.0, 31.0, 0.0, 15.0, 2.0, 1.0, 0.0, 45.0, 6.0, 3.0, 0.0, 3.0, 2.0, 46.0, 229.0, 29.0, 26.0, 6.0, 2.0, 25.0, 1.0, 1.0, 145.0, 31.0, 17.0, 155.0, 24.0, 160.0, 60.0, 34.0, 80.0, 16.0, 45.0, 12.0, 53.0, 84.0, 169.0, 32.0, 118.0, 29.0, 73.0, 11.0, 342.0, 187.0, 45.0, 84.0, 79.0, 34.0, 60.0, 57.0, 30.0, 93.0, 71.0, 33.0, 70.0, 29.0, 6.0, 239.0, 64.0, 155.0, 73.0, 21.0, 167.0, 65.0, 29.0, 26.0, 73.0, 32.0, 18.0, 79.0, 148.0, 18.0, 66.0, 13.0, 19.0, 5.0, 43.0, 26.0, 6.0, 10.0, 27.0, 8.0, 44.0, 10.0, 4.0, 30.0, 11.0, 23.0, 8.0, 4.0, 68.0, 6.0, 14.0, 6.0, 6.0, 1.0, 23.0, 3.0, 2.0, 77.0, 6.0, 6.0, 3.0, 4.0, 2.0, 2.0, 157.0, 75.0, 7.0, 12.0, 3.0, 65.0, 85.0, 47.0, 28.0, 110.0, 7.0, 98.0, 148.0, 32.0, 17.0, 11.0, 2.0, 2.0, 3.0, 6.0, 7.0, 8.0, 7.0, 6.0, 84.0, 39.0, 27.0, 11.0, 4.0, 5.0, 4.0, 4.0, 0.0, 3.0, 0.0, 6.0, 1.0, 53.0, 0.0, 13.0, 6.0, 3.0, 6.0, 16.0, 85.0, 19.0, 65.0, 1.0, 16.0, 5.0, 21.0, 9.0, 1.0, 58.0, 7.0, 3.0, 28.0, 12.0, 16.0, 0.0, 0.0, 1.0, 2.0, 4.0, 1.0, 1.0, 7.0, 0.0, 144.0, 0.0, 6.0, 146.0, 105.0, 21.0, 1.0, 8.0, 48.0, 2.0, 0.0, 77.0, 4.0, 2.0, 91.0, 0.0, 6.0, 37.0, 28.0, 23.0, 1.0, 0.0, 21.0, 2.0, 9.0, 8.0, 2.0, 2.0, 37.0, 22.0, 7.0, 0.0, 21.0, 3.0, 86.0, 21.0, 4.0, 2.0, 113.0, 10.0, 30.0, 0.0, 0.0, 10.0, 51.0, 21.0, 2.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 8.0, 1.0, 299.0, 100.0, 44.0, 17.0, 8.0, 4.0, 30.0, 21.0, 6.0, 0.0, 1.0, 2.0, 9.0, 8.0, 3.0, 8.0, 5.0, 0.0, 18.0, 22.0, 26.0, 3.0, 4.0, 29.0, 12.0, 14.0, 2.0, 17.0, 0.0, 1.0, 7.0, 1.0, 64.0, 6.0, 16.0, 8.0, 26.0, 0.0, 40.0, 26.0, 3.0, 46.0, 14.0, 6.0, 10.0, 1.0, 4.0, 246.0, 7.0, 14.0, 10.0, 0.0, 5.0, 2.0, 5.0, 43.0, 47.0, 6.0, 25.0, 4.0, 6.0, 4.0, 1.0, 4.0, 6.0, 8.0, 4.0, 10.0, 45.0, 34.0, 11.0, 4.0, 92.0, 0.0, 5.0, 43.0, 2.0, 1.0, 5.0, 6.0, 0.0, 1.0, 6.0, 9.0, 6.0, 0.0, 8.0, 3.0, 22.0, 15.0, 5.0, 38.0, 36.0, 9.0, 4.0, 15.0, 0.0, 53.0, 39.0, 64.0, 25.0, 36.0, 64.0, 2.0, 11.0, 0.0, 3.0, 4.0, 2.0, 5.0, 24.0, 18.0, 11.0, 10.0, 5.0, 29.0, 14.0, 2.0, 18.0, 298.0, 107.0, 132.0, 18.0, 7.0, 2.0, 19.0, 9.0, 1.0, 144.0, 9.0, 17.0, 2.0, 3.0, 5.0, 2.0, 110.0, 25.0, 89.0, 22.0, 22.0, 20.0, 33.0, 0.0, 2.0, 8.0, 0.0, 4.0, 141.0, 12.0, 138.0, 11.0, 19.0, 99.0, 21.0, 250.0, 46.0, 47.0, 34.0, 93.0, 90.0, 12.0, 125.0, 60.0, 3.0, 115.0, 38.0, 57.0, 15.0, 14.0, 11.0, 299.0, 175.0, 99.0, 90.0, 30.0, 0.0, 19.0, 2.0, 8.0, 60.0, 10.0, 68.0, 27.0, 151.0, 27.0, 7.0, 38.0, 233.0, 59.0, 54.0, 146.0, 170.0, 12.0, 32.0, 51.0, 195.0, 336.0, 69.0, 95.0, 6.0, 3.0, 128.0, 57.0, 46.0, 3.0, 40.0, 27.0, 14.0, 34.0, 9.0, 27.0, 22.0, 78.0, 40.0, 71.0, 122.0, 13.0, 64.0, 382.0, 410.0, 224.0, 134.0, 286.0, 211.0, 67.0, 417.0, 170.0, 225.0, 209.0, 135.0, 332.0, 414.0, 261.0, 143.0, 275.0, 410.0, 201.0, 73.0, 94.0, 8.0, 165.0, 48.0, 60.0, 92.0, 38.0, 14.0, 46.0, 66.0, 41.0, 127.0, 8.0, 159.0, 165.0, 204.0, 174.0, 53.0, 95.0, 227.0, 96.0, 90.0, 55.0, 10.0, 72.0, 71.0, 50.0, 46.0, 36.0, 25.0, 12.0, 73.0, 40.0, 260.0, 10.0, 63.0, 52.0, 45.0, 11.0, 140.0, 304.0, 121.0, 19.0, 9.0, 11.0, 2.0, 84.0, 14.0, 85.0, 100.0, 34.0, 89.0, 159.0, 113.0, 130.0, 56.0, 111.0, 182.0, 25.0, 24.0, 7.0, 44.0, 84.0, 195.0, 188.0, 32.0, 7.0, 89.0, 83.0, 35.0, 74.0, 33.0, 36.0, 38.0, 45.0, 25.0, 25.0, 25.0, 115.0, 27.0, 12.0, 15.0, 5.0, 109.0, 372.0, 449.0, 74.0, 225.0, 17.0, 2.0, 163.0, 30.0, 86.0, 77.0, 60.0, 70.0, 5.0, 2.0, 72.0, 9.0, 57.0, 1.0, 2.0, 22.0, 0.0, 103.0, 25.0, 145.0, 3.0, 37.0, 31.0, 31.0, 13.0, 8.0, 67.0, 111.0, 16.0, 52.0, 11.0, 4.0, 13.0, 12.0, 54.0, 5.0, 76.0, 7.0, 24.0, 177.0, 118.0, 24.0, 37.0, 22.0, 21.0, 19.0, 21.0, 13.0, 64.0, 57.0, 175.0, 18.0, 37.0, 95.0, 7.0, 100.0, 6.0, 8.0, 128.0, 84.0, 2.0, 93.0, 55.0, 237.0, 53.0, 0.0, 124.0, 55.0, 89.0, 28.0, 24.0, 56.0, 39.0, 10.0, 8.0, 284.0, 22.0, 6.0, 4.0, 43.0, 3.0, 37.0, 58.0, 55.0, 10.0, 14.0, 20.0, 72.0, 19.0, 27.0, 10.0, 13.0, 6.0, 16.0, 64.0, 11.0, 27.0, 32.0, 178.0, 3.0, 9.0, 198.0, 16.0, 9.0, 5.0, 64.0, 85.0, 4.0, 16.0, 5.0, 29.0, 30.0, 41.0, 15.0, 103.0, 6.0, 23.0, 5.0, 8.0, 93.0, 109.0, 7.0, 90.0, 40.0, 185.0, 9.0, 123.0, 146.0, 660.0, 74.0, 53.0, 136.0, 298.0, 9.0, 40.0, 65.0, 34.0, 122.0, 4.0, 4.0, 25.0, 167.0, 48.0, 224.0, 150.0, 27.0, 93.0, 91.0, 2.0, 12.0, 16.0, 13.0, 116.0, 10.0, 7.0, 9.0, 86.0, 14.0, 179.0, 72.0, 159.0, 19.0, 19.0, 8.0, 198.0, 7.0, 14.0, 276.0, 250.0, 23.0, 92.0, 125.0, 76.0, 23.0, 8.0, 200.0, 99.0, 14.0, 6.0, 19.0, 16.0, 11.0, 254.0, 6.0, 53.0, 49.0, 117.0, 239.0, 38.0, 10.0, 13.0, 305.0, 123.0, 196.0, 301.0, 157.0, 295.0, 499.0, 37.0, 358.0, 68.0, 121.0, 1.0, 10.0, 7.0, 26.0, 60.0, 28.0, 13.0, 64.0, 2.0, 61.0, 45.0, 223.0, 382.0, 31.0, 166.0, 512.0, 208.0, 175.0, 27.0, 42.0, 25.0, 90.0, 98.0, 19.0, 213.0, 116.0, 68.0, 2.0, 197.0, 13.0, 218.0, 166.0, 420.0, 107.0, 19.0, 126.0, 103.0, 8.0, 344.0, 67.0, 94.0, 232.0, 331.0, 400.0, 478.0, 175.0, 154.0, 39.0, 23.0, 422.0, 900.0, 502.0, 65.0, 56.0, 29.0, 26.0, 47.0, 25.0, 810.0, 126.0, 30.0, 40.0, 178.0, 289.0, 14.0, 651.0, 347.0, 18.0, 258.0, 378.0, 405.0, 90.0, 478.0, 585.0, 297.0, 54.0, 13.0, 15.0, 115.0, 968.0, 8113.0, 4155.0, 2821.0, 1513.0, 1103.0, 2516.0, 3785.0, 2449.0, 1720.0, 2479.0, 12114.0, 4531.0, 5870.0, 4658.0, 3712.0, 11746.0, 1964.0, 2133.0, 4035.0, 755.0, 5529.0, 3049.0, 2592.0, 12034.0, 2069.0, 4354.0, 2448.0, 1361.0, 2022.0, 1388.0, 6302.0, 3850.0, 3503.0, 2673.0, 5376.0, 2592.0, 3812.0, 2758.0, 1599.0, 3866.0, 4713.0, 5211.0, 2909.0, 2833.0, 1943.0, 2524.0, 4640.0, 2402.0, 3360.0, 2863.0, 2274.0, 1921.0, 3537.0, 3924.0, 1648.0, 2047.0, 2923.0, 2690.0, 1881.0, 1783.0, 961.0, 2160.0, 4450.0, 974.0, 3142.0, 1772.0, 1597.0, 2196.0, 1789.0, 3013.0, 1202.0, 1545.0, 1844.0, 776.0, 1441.0, 2668.0, 669.0, 1699.0, 946.0, 1965.0, 1161.0, 404.0, 1544.0, 1202.0, 2362.0, 1049.0, 2207.0, 1582.0, 1977.0, 1311.0, 2193.0, 797.0, 161.0, 656.0, 1858.0, 1019.0, 1156.0, 863.0, 623.0, 992.0, 892.0, 2140.0, 1698.0, 700.0, 1545.0, 1433.0, 236.0, 580.0, 1238.0, 1190.0, 715.0, 3380.0, 3886.0, 644.0, 657.0, 751.0, 95.0, 1087.0, 2398.0, 1224.0, 729.0, 493.0, 592.0, 417.0, 819.0, 797.0, 1193.0, 412.0, 106.0, 270.0, 1287.0, 98.0, 717.0, 392.0, 194.0, 146.0, 104.0, 145.0, 154.0, 1312.0, 113.0, 212.0, 108.0, 1190.0, 2016.0, 3908.0, 2861.0, 1519.0, 995.0, 1143.0, 8749.0, 2233.0, 4573.0, 2948.0, 2473.0, 4387.0, 6292.0, 4245.0, 2250.0, 2736.0, 1853.0, 2058.0, 702.0, 7188.0, 4485.0, 4783.0, 3773.0, 2091.0, 2278.0, 2843.0, 2032.0, 2009.0, 1265.0, 1258.0, 1014.0, 653.0, 3553.0, 573.0, 2827.0, 1656.0, 2335.0, 957.0, 1252.0, 745.0, 4959.0, 1469.0, 1339.0, 1072.0, 1365.0, 867.0, 1790.0, 8319.0, 3598.0, 2827.0, 6152.0, 3016.0, 3767.0, 1205.0, 2692.0, 2422.0, 1648.0, 2344.0, 1273.0, 1924.0, 2936.0, 1339.0, 1299.0, 1081.0, 88.0, 2199.0, 1018.0, 2089.0, 1229.0, 3233.0, 5438.0, 1013.0, 1078.0, 1833.0, 1822.0, 938.0, 640.0, 703.0, 560.0, 391.0, 830.0, 775.0, 688.0, 277.0, 864.0, 91.0, 249.0, 493.0, 535.0, 835.0, 764.0, 782.0, 579.0, 2003.0, 497.0, 911.0, 1690.0, 295.0, 1100.0, 335.0, 880.0, 484.0, 71.0, 865.0, 444.0, 500.0, 651.0, 361.0, 366.0, 1410.0, 846.0, 1103.0, 960.0, 610.0, 518.0, 2270.0, 2419.0, 823.0, 1527.0, 1287.0, 1735.0, 3879.0, 2362.0, 1136.0, 958.0, 1699.0, 2956.0, 1641.0, 1767.0, 814.0, 769.0, 2515.0, 1499.0, 1072.0, 1386.0, 763.0, 1084.0, 2539.0, 4540.0, 1598.0, 1795.0, 3906.0, 2593.0, 2757.0, 2879.0, 1524.0, 2266.0, 858.0, 1488.0, 1091.0, 1973.0, 1667.0, 1002.0, 285.0, 901.0, 2318.0, 1813.0, 3486.0, 1514.0, 976.0, 570.0, 2167.0, 1434.0, 1989.0, 1857.0, 1242.0, 816.0, 1204.0, 1528.0, 1105.0, 894.0, 1098.0, 1678.0, 733.0, 1755.0, 2682.0, 2043.0, 1554.0, 2186.0, 4348.0, 1263.0, 3330.0, 1831.0, 1798.0, 912.0, 1156.0, 1183.0, 1173.0, 1680.0, 4789.0, 1642.0, 3345.0, 2011.0, 1979.0, 1234.0, 2750.0, 2561.0, 1365.0, 1701.0, 480.0, 927.0, 2443.0, 1415.0, 2159.0, 343.0, 1561.0, 869.0, 733.0, 184.0, 1581.0, 1397.0, 1363.0, 1061.0, 1406.0, 1179.0, 1501.0, 2085.0, 1326.0, 1523.0, 2145.0, 478.0, 525.0, 342.0, 759.0, 1683.0, 2010.0, 1642.0, 2624.0, 6795.0, 2196.0, 3116.0, 3373.0, 5841.0, 2185.0, 2878.0, 1934.0, 7365.0, 4198.0, 1083.0, 906.0, 8282.0, 9425.0, 2375.0, 6117.0, 5850.0, 44028.0, 28515.0, 7256.0, 5389.0, 4141.0, 1868.0, 2563.0, 1028.0, 1584.0, 1928.0, 1732.0, 1104.0, 1427.0, 1456.0, 2081.0, 6858.0]})}, {'time': ('2016-12-28 16:31:00', {'close': [2245.75, 2245.75, 2245.75, 2245.5, 2245.75, 2245.75, 2245.75, 2245.75, 2245.75, 2245.75, 2245.75, 2245.75, 2245.75, 2245.75, 2245.75, 2246.0, 2245.75, 2246.0, 2245.75, 2245.75, 2245.75, 2246.0, 2245.5, 2245.75, 2245.5, 2245.5, 2245.0, 2245.5, 2245.25, 2244.75], 'volume': [1106.0, 86.0, 446.0, 142.0, 161.0, 108.0, 92.0, 14.0, 86.0, 273.0, 35.0, 155.0, 57.0, 48.0, 308.0, 120.0, 141.0, 70.0, 85.0, 50.0, 129.0, 93.0, 166.0, 202.0, 91.0, 577.0, 218.0, 179.0, 283.0, 632.0]})}, {'time': ('2016-12-28 18:01:00', {'close': [2246.25, 2246.25, 2246.25, 2246.75, 2246.5, 2246.5, 2246.5, 2246.75, 2246.75, 2246.5, 2246.75, 2246.5, 2246.5, 2246.5, 2246.75, 2247.0, 2247.0, 2247.0, 2246.75, 2246.75, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.75, 2246.5, 2246.75, 2246.75, 2246.75, 2246.75, 2247.0, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2247.0, 2246.75, 2246.75, 2247.0, 2247.0, 2247.0, 2247.0, 2247.0, 2247.0, 2246.5, 2247.0, 2247.0, 2247.0, 2246.75, 2246.5, 2246.5, 2246.75, 2247.0, 2246.75, 2247.0, 2246.5, 2246.75, 2246.5, 2246.5, 2246.25, 2246.25, 2246.5, 2246.0, 2246.0, 2246.0, 2246.25, 2246.25, 2246.0, 2246.0, 2246.0, 2245.75, 2245.75, 2246.0, 2245.75, 2246.0, 2246.25, 2246.5, 2246.0, 2246.25, 2246.25, 2246.0, 2246.0, 2246.0, 2246.25, 2246.0, 2246.25, 2246.25, 2246.25, 2246.25, 2246.25, 2246.25, 2246.25, 2246.25, 2246.25, 2246.5, 2246.0, 2246.25, 2246.25, 2246.25, 2246.25, 2246.0, 2246.0, 2246.0, 2245.5, 2245.5, 2245.75, 2245.5, 2245.5, 2245.5, 2245.75, 2245.5, 2245.5, 2245.5, 2245.5, 2246.0, 2246.0, 2245.75, 2246.0, 2246.25, 2246.25, 2246.5, 2246.25, 2246.25, 2246.25, 2246.25, 2246.0, 2246.25, 2246.25, 2246.0, 2246.25, 2246.5, 2246.0, 2245.75, 2245.75, 2245.25, 2245.25, 2245.0, 2245.25, 2245.0, 2245.0, 2245.25, 2245.0, 2245.25, 2245.0, 2244.75, 2244.75, 2244.75, 2244.75, 2245.0, 2245.25, 2245.25, 2245.0, 2245.25, 2245.25, 2245.5, 2245.5, 2245.75, 2246.0, 2245.75, 2246.25, 2246.0, 2246.25, 2246.0, 2246.25, 2246.5, 2246.5, 2246.75, 2246.5, 2246.5, 2246.5, 2246.5, 2246.75, 2246.5, 2246.75, 2246.75, 2246.75, 2246.25, 2246.0, 2246.0, 2246.5, 2246.25, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.75, 2247.0, 2246.5, 2246.75, 2246.75, 2246.5, 2246.5, 2246.25, 2246.5, 2246.5, 2246.75, 2246.75, 2246.5, 2246.75, 2246.25, 2246.5, 2246.75, 2246.75, 2246.5, 2246.75, 2246.75, 2246.75, 2246.5, 2246.5, 2246.75, 2246.75, 2246.5, 2246.75, 2246.5, 2246.75, 2246.75, 2246.75, 2246.75, 2246.5, 2246.75, 2246.5, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.5, 2246.75, 2246.75, 2246.75, 2246.5, 2246.75, 2246.75, 2246.5, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.5, 2246.25, 2246.5, 2246.25, 2246.5, 2246.25, 2246.5, 2246.5, 2246.5, 2246.5, 2246.25, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.75, 2246.75, 2247.0, 2246.75, 2246.5, 2246.75, 2246.75, 2246.75, 2246.5, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2247.0, 2246.75, 2246.75, 2246.5, 2246.75, 2246.75, 2246.75, 2247.0, 2247.0, 2247.0, 2247.0, 2246.75, 2247.0, 2247.0, 2247.0, 2246.75, 2246.75, 2246.5, 2246.5, 2246.5, 2246.75, 2246.5, 2246.5, 2246.75, 2246.75, 2246.5, 2246.75, 2246.5, 2246.75, 2246.75, 2246.75, 2246.5, 2246.5, 2246.25, 2246.25, 2246.25, 2246.5, 2246.25, 2246.5, 2246.5, 2246.75, 2246.5, 2246.5, 2246.5, 2246.75, 2246.75, 2246.5, 2246.75, 2246.75, 2246.5, 2246.75, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.25, 2246.25, 2246.25, 2246.5, 2246.5, 2246.5, 2246.5, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.5, 2246.75, 2247.0, 2247.0, 2247.0, 2247.0, 2247.0, 2247.0, 2246.75, 2247.0, 2247.0, 2246.75, 2247.0, 2246.75, 2246.75, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.75, 2246.5, 2246.5, 2246.5, 2246.5, 2246.25, 2246.25, 2246.25, 2246.25, 2246.25, 2246.0, 2246.25, 2246.0, 2246.0, 2246.0, 2246.0, 2246.25, 2246.25, 2246.25, 2246.25, 2246.25, 2246.25, 2246.5, 2246.5, 2246.5, 2246.25, 2246.5, 2246.5, 2246.5, 2246.5, 2246.25, 2246.25, 2246.25, 2246.25, 2246.0, 2246.0, 2246.25, 2246.25, 2246.25, 2246.0, 2246.0, 2245.75, 2245.5, 2245.5, 2245.75, 2245.75, 2246.0, 2245.75, 2246.0, 2246.0, 2246.0, 2245.75, 2245.5, 2245.75, 2245.5, 2245.5, 2245.5, 2245.0, 2245.0, 2245.0, 2245.0, 2245.0, 2245.0, 2245.0, 2245.25, 2245.0, 2245.0, 2245.25, 2245.0, 2245.0, 2245.25, 2245.0, 2245.0, 2245.25, 2245.5, 2245.75, 2245.75, 2245.75, 2245.75, 2245.75, 2245.5, 2245.75, 2246.25, 2246.25, 2246.5, 2246.5, 2246.25, 2246.0, 2245.75, 2246.0, 2246.0, 2246.0, 2246.0, 2245.75, 2246.25, 2246.0, 2246.0, 2246.5, 2246.25, 2246.25, 2246.25, 2246.25, 2246.25, 2246.25, 2246.5, 2246.0, 2245.75, 2245.75, 2246.0, 2245.5, 2245.5, 2245.25, 2245.5, 2245.75, 2245.5, 2245.25, 2245.0, 2244.5, 2244.75, 2244.5, 2244.5, 2244.5, 2244.75, 2244.75, 2244.5, 2244.25, 2244.75, 2244.5, 2244.5, 2244.5, 2244.5, 2244.75, 2244.75, 2244.75, 2244.5, 2245.0, 2245.0, 2245.25, 2245.5, 2245.5, 2245.75, 2245.75, 2245.5, 2245.25, 2245.25, 2245.0, 2245.25, 2245.5, 2245.5, 2245.5, 2245.25, 2245.25, 2244.75, 2244.5, 2244.5, 2244.5, 2244.25, 2244.5, 2244.0, 2244.0, 2243.25, 2243.25, 2243.25, 2243.0, 2243.5, 2243.25, 2243.75, 2244.25, 2244.5, 2244.75, 2244.25, 2244.25, 2244.75, 2244.25, 2245.0, 2245.0, 2245.0, 2245.0, 2244.75, 2244.75, 2244.5, 2244.75, 2244.75, 2244.75, 2245.0, 2244.25, 2244.75, 2245.25, 2245.25, 2244.75, 2244.75, 2244.75, 2245.25, 2244.25, 2244.5, 2244.5, 2244.75, 2244.25, 2244.5, 2244.5, 2244.75, 2244.75, 2244.5, 2244.25, 2244.5, 2244.25, 2244.5, 2244.5, 2244.25, 2244.5, 2244.75, 2244.5, 2244.5, 2244.25, 2244.0, 2243.75, 2244.5, 2244.5, 2244.5, 2244.75, 2245.0, 2244.75, 2244.75, 2244.75, 2244.75, 2244.75, 2244.75, 2245.0, 2245.25, 2245.25, 2244.75, 2244.5, 2244.5, 2244.75, 2245.0, 2245.0, 2245.25, 2245.0, 2245.0, 2245.0, 2245.75, 2245.75, 2245.5, 2245.75, 2245.5, 2245.5, 2245.5, 2245.75, 2246.0, 2246.0, 2246.25, 2245.75, 2246.0, 2246.0, 2246.5, 2246.5, 2246.0, 2246.0, 2246.0, 2246.0, 2245.75, 2246.0, 2245.5, 2245.25, 2245.25, 2245.0, 2245.0, 2245.0, 2244.75, 2244.75, 2244.5, 2244.5, 2244.75, 2245.0, 2245.0, 2245.0, 2244.75, 2245.0, 2244.75, 2244.75, 2245.0, 2245.0, 2245.0, 2245.0, 2245.0, 2245.25, 2245.25, 2244.75, 2244.75, 2244.75, 2244.75, 2244.75, 2244.5, 2245.0, 2245.0, 2245.0, 2245.0, 2245.0, 2245.0, 2245.25, 2245.0, 2245.0, 2245.0, 2245.25, 2245.25, 2245.25, 2245.0, 2245.25, 2245.75, 2245.5, 2245.5, 2245.75, 2245.75, 2245.75, 2246.0, 2246.0, 2246.25, 2246.0, 2246.0, 2246.0, 2246.0, 2245.5, 2245.5, 2245.5, 2245.5, 2245.75, 2246.0, 2246.0, 2246.25, 2246.0, 2246.25, 2246.75, 2246.75, 2246.75, 2246.5, 2246.5, 2246.5, 2246.25, 2246.5, 2246.5, 2246.25, 2246.75, 2246.75, 2246.75, 2246.5, 2246.25, 2246.5, 2246.5, 2246.25, 2246.5, 2246.5, 2246.25, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.25, 2246.25, 2246.5, 2246.0, 2246.0, 2246.25, 2246.0, 2246.0, 2246.0, 2246.25, 2246.25, 2246.0, 2246.0, 2246.25, 2246.25, 2246.0, 2246.0, 2245.75, 2246.0, 2245.75, 2245.5, 2245.75, 2245.75, 2245.5, 2245.5, 2245.5, 2245.75, 2245.75, 2245.5, 2245.75, 2245.5, 2245.5, 2245.75, 2245.75, 2245.5, 2245.75, 2245.75, 2245.75, 2245.5, 2245.5, 2245.75, 2245.75, 2245.5, 2245.75, 2245.25, 2245.5, 2245.5, 2245.25, 2245.5, 2245.5, 2245.5, 2245.75, 2245.75, 2245.75, 2245.75, 2246.25, 2246.25, 2246.25, 2246.5, 2246.25, 2246.0, 2246.25, 2246.25, 2246.25, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.25, 2246.5, 2246.5, 2246.5, 2246.25, 2246.25, 2246.25, 2246.5, 2246.5, 2246.5, 2246.75, 2246.75, 2247.0, 2247.0, 2247.0, 2247.0, 2247.25, 2247.25, 2247.25, 2247.25, 2247.25, 2246.75, 2247.0, 2246.75, 2247.0, 2246.75, 2246.75, 2247.0, 2246.75, 2247.0, 2247.25, 2247.0, 2247.0, 2247.0, 2247.0, 2247.0, 2247.0, 2247.0, 2247.5, 2247.0, 2247.25, 2247.25, 2247.25, 2247.25, 2247.0, 2247.25, 2247.5, 2247.75, 2247.75, 2247.75, 2247.75, 2247.75, 2247.75, 2247.5, 2247.5, 2247.5, 2247.5, 2247.5, 2247.5, 2247.0, 2246.75, 2246.25, 2246.25, 2246.75, 2246.75, 2247.0, 2247.0, 2247.25, 2246.5, 2247.0, 2246.75, 2246.5, 2246.5, 2246.0, 2246.0, 2246.0, 2245.75, 2245.75, 2245.75, 2245.75, 2246.0, 2246.75, 2246.5, 2246.75, 2246.75, 2246.75, 2246.5, 2247.0, 2246.5, 2246.5, 2246.5, 2246.25, 2246.0, 2246.0, 2246.25, 2246.25, 2246.25, 2246.25, 2246.0, 2245.75, 2245.75, 2245.75, 2245.5, 2245.5, 2246.0, 2245.75, 2246.25, 2246.5, 2246.5, 2246.5, 2246.5, 2246.75, 2246.75, 2246.75, 2246.5, 2246.25, 2246.5, 2246.25, 2246.75, 2246.75, 2246.5, 2246.75, 2246.0, 2246.25, 2246.25, 2246.25, 2245.25, 2245.75, 2246.75, 2247.75, 2248.75, 2248.5, 2248.75, 2249.0, 2248.75, 2249.0, 2249.0, 2249.0, 2249.25, 2248.5, 2248.75, 2248.25, 2247.75, 2247.25, 2247.25, 2247.25, 2246.0, 2245.0, 2245.75, 2246.0, 2246.5, 2246.5, 2246.5, 2246.5, 2246.75, 2246.0, 2246.5, 2246.75, 2247.75, 2248.5, 2248.0, 2248.25, 2248.0, 2247.25, 2246.75, 2245.75, 2246.25, 2246.25, 2246.5, 2246.0, 2246.25, 2247.0, 2246.75, 2247.0, 2246.75, 2247.5, 2248.0, 2247.5, 2247.75, 2247.5, 2247.0, 2246.75, 2247.25, 2248.75, 2248.0, 2247.5, 2247.5, 2248.0, 2247.5, 2248.0, 2248.0, 2247.25, 2247.0, 2247.5, 2247.0, 2246.75, 2246.5, 2246.25, 2245.5, 2246.0, 2245.5, 2246.0, 2245.5, 2245.5, 2245.75, 2245.0, 2245.0, 2244.25, 2244.0, 2244.5, 2243.75, 2243.75, 2244.25, 2244.5, 2244.25, 2244.5, 2244.25, 2244.25, 2244.5, 2244.25, 2243.5, 2243.75, 2243.5, 2244.0, 2243.0, 2243.5, 2244.0, 2243.75, 2243.75, 2244.5, 2244.5, 2244.0, 2244.25, 2244.0, 2243.5, 2242.75, 2243.5, 2244.0, 2244.0, 2244.0, 2243.75, 2243.25, 2243.75, 2244.5, 2244.25, 2244.0, 2244.5, 2244.25, 2244.5, 2244.75, 2244.5, 2244.75, 2244.5, 2245.0, 2244.75, 2244.5, 2244.0, 2243.5, 2243.75, 2244.0, 2243.75, 2243.5, 2243.75, 2243.75, 2243.75, 2244.5, 2243.75, 2243.25, 2243.0, 2242.25, 2241.5, 2242.75, 2242.5, 2242.75, 2243.0, 2242.5, 2242.25, 2242.5, 2242.25, 2242.5, 2242.25, 2242.5, 2242.25, 2242.25, 2242.0, 2241.5, 2241.25, 2241.0, 2240.75, 2240.0, 2240.0, 2240.0, 2240.75, 2241.0, 2241.25, 2241.5, 2242.0, 2241.5, 2242.0, 2242.0, 2241.75, 2241.5, 2242.5, 2242.5, 2242.5, 2243.0, 2242.75, 2243.0, 2243.5, 2242.5, 2242.75, 2242.5, 2242.5, 2242.75, 2242.5, 2242.5, 2242.5, 2242.25, 2242.75, 2243.25, 2243.5, 2243.25, 2243.0, 2242.75, 2242.75, 2243.25, 2243.0, 2243.75, 2244.0, 2244.25, 2244.25, 2244.25, 2244.25, 2244.0, 2244.0, 2243.5, 2243.5, 2242.5, 2242.75, 2243.0, 2242.5, 2242.25, 2242.25, 2242.25, 2242.25, 2242.25, 2242.25, 2242.25, 2242.0, 2242.5, 2242.5, 2242.5, 2242.75, 2242.75, 2242.5, 2242.75, 2242.75, 2242.75, 2242.75, 2242.5, 2242.5, 2242.25, 2242.5, 2242.75, 2242.75, 2243.0, 2242.75, 2242.75, 2242.25, 2241.75, 2241.75, 2242.0, 2241.75, 2241.5, 2241.5, 2241.25, 2241.25, 2241.0, 2241.0, 2241.25, 2241.0, 2240.75, 2241.0, 2241.0, 2241.0, 2241.25, 2240.75, 2240.75, 2241.5, 2242.0, 2242.0, 2241.75, 2242.25, 2242.0, 2241.5, 2242.25, 2242.75, 2243.0, 2244.0, 2244.5, 2245.0, 2244.75, 2245.0, 2244.75, 2244.5, 2244.75, 2244.75, 2244.75, 2244.75, 2244.75, 2244.5, 2244.75, 2244.5, 2244.75, 2244.5, 2244.75, 2245.0, 2245.0, 2245.5, 2246.0, 2245.5, 2245.75, 2245.25, 2245.25, 2244.75, 2244.5, 2244.25, 2244.5, 2244.25, 2244.5, 2244.0, 2243.5, 2243.0, 2243.5, 2243.5, 2243.5, 2243.5, 2243.5, 2243.0, 2243.0, 2243.0, 2243.25, 2244.0, 2244.5, 2244.75, 2244.75, 2244.0, 2244.25, 2244.0, 2244.5, 2244.5, 2244.25, 2244.5, 2244.25, 2244.25, 2244.25, 2244.25, 2244.25, 2244.25, 2244.75, 2244.5, 2244.0, 2244.0, 2243.75, 2243.5, 2243.25, 2243.25, 2243.5, 2243.5, 2244.0, 2243.75, 2244.25, 2244.25, 2244.25, 2244.5, 2244.0, 2244.5, 2243.75, 2244.25, 2244.5, 2244.5, 2244.75, 2244.5, 2244.25, 2244.25, 2244.5, 2244.25, 2244.0, 2244.75, 2244.75, 2244.5, 2244.25, 2244.0, 2243.25, 2243.5, 2243.5, 2243.25, 2243.5, 2243.75, 2243.5, 2243.5, 2243.5, 2243.5, 2244.0, 2243.75, 2244.0, 2243.5, 2243.75, 2244.25, 2244.25, 2244.0, 2244.0, 2243.75, 2244.0, 2244.5, 2244.5, 2244.5, 2245.0, 2244.5, 2244.5, 2244.75, 2245.0, 2245.0, 2245.0, 2245.25, 2245.25, 2245.0, 2245.5, 2246.0, 2245.75, 2246.0, 2245.25, 2245.25], 'volume': [712.0, 292.0, 173.0, 144.0, 60.0, 44.0, 69.0, 136.0, 30.0, 6.0, 10.0, 32.0, 10.0, 106.0, 23.0, 33.0, 58.0, 100.0, 22.0, 24.0, 158.0, 21.0, 9.0, 60.0, 24.0, 20.0, 9.0, 28.0, 7.0, 51.0, 29.0, 29.0, 47.0, 5.0, 13.0, 27.0, 7.0, 22.0, 18.0, 11.0, 10.0, 16.0, 29.0, 61.0, 64.0, 8.0, 20.0, 89.0, 41.0, 15.0, 25.0, 71.0, 38.0, 35.0, 15.0, 30.0, 20.0, 16.0, 52.0, 42.0, 70.0, 32.0, 114.0, 152.0, 14.0, 299.0, 177.0, 136.0, 123.0, 80.0, 26.0, 23.0, 54.0, 80.0, 37.0, 34.0, 25.0, 169.0, 85.0, 70.0, 62.0, 58.0, 31.0, 104.0, 89.0, 46.0, 6.0, 24.0, 78.0, 34.0, 15.0, 8.0, 36.0, 4.0, 30.0, 7.0, 13.0, 48.0, 109.0, 2.0, 11.0, 13.0, 4.0, 66.0, 33.0, 9.0, 149.0, 78.0, 3.0, 13.0, 15.0, 17.0, 3.0, 121.0, 10.0, 18.0, 16.0, 86.0, 6.0, 7.0, 187.0, 60.0, 109.0, 97.0, 37.0, 45.0, 8.0, 12.0, 4.0, 3.0, 12.0, 63.0, 45.0, 30.0, 17.0, 263.0, 4.0, 247.0, 36.0, 116.0, 24.0, 104.0, 92.0, 96.0, 60.0, 13.0, 112.0, 144.0, 121.0, 22.0, 302.0, 162.0, 86.0, 170.0, 47.0, 139.0, 441.0, 209.0, 62.0, 81.0, 89.0, 11.0, 193.0, 79.0, 11.0, 42.0, 21.0, 69.0, 91.0, 56.0, 76.0, 83.0, 43.0, 7.0, 85.0, 106.0, 35.0, 9.0, 41.0, 79.0, 87.0, 4.0, 46.0, 92.0, 56.0, 17.0, 6.0, 6.0, 41.0, 4.0, 103.0, 22.0, 3.0, 4.0, 67.0, 17.0, 34.0, 48.0, 23.0, 12.0, 45.0, 5.0, 7.0, 61.0, 19.0, 4.0, 2.0, 9.0, 5.0, 8.0, 5.0, 14.0, 16.0, 13.0, 6.0, 8.0, 14.0, 27.0, 13.0, 3.0, 3.0, 3.0, 7.0, 8.0, 5.0, 23.0, 61.0, 14.0, 8.0, 9.0, 2.0, 5.0, 5.0, 13.0, 38.0, 30.0, 3.0, 4.0, 57.0, 6.0, 58.0, 53.0, 19.0, 3.0, 4.0, 16.0, 6.0, 5.0, 5.0, 10.0, 42.0, 23.0, 8.0, 8.0, 13.0, 3.0, 17.0, 4.0, 3.0, 13.0, 6.0, 3.0, 5.0, 5.0, 3.0, 78.0, 5.0, 153.0, 67.0, 59.0, 12.0, 6.0, 25.0, 82.0, 154.0, 7.0, 4.0, 32.0, 22.0, 37.0, 206.0, 42.0, 20.0, 3.0, 5.0, 6.0, 10.0, 23.0, 57.0, 131.0, 15.0, 64.0, 21.0, 7.0, 4.0, 2.0, 4.0, 10.0, 92.0, 8.0, 5.0, 12.0, 6.0, 2.0, 144.0, 37.0, 83.0, 38.0, 7.0, 42.0, 34.0, 51.0, 47.0, 7.0, 7.0, 12.0, 8.0, 37.0, 10.0, 36.0, 12.0, 6.0, 2.0, 4.0, 28.0, 7.0, 6.0, 19.0, 6.0, 10.0, 15.0, 37.0, 33.0, 8.0, 9.0, 17.0, 20.0, 11.0, 4.0, 5.0, 11.0, 12.0, 16.0, 15.0, 20.0, 14.0, 59.0, 16.0, 5.0, 142.0, 139.0, 7.0, 15.0, 4.0, 5.0, 9.0, 5.0, 19.0, 66.0, 5.0, 13.0, 12.0, 115.0, 51.0, 23.0, 3.0, 14.0, 4.0, 5.0, 52.0, 46.0, 20.0, 0.0, 5.0, 1.0, 75.0, 237.0, 6.0, 1.0, 113.0, 21.0, 8.0, 5.0, 6.0, 71.0, 39.0, 22.0, 26.0, 6.0, 8.0, 40.0, 3.0, 6.0, 100.0, 6.0, 4.0, 2.0, 15.0, 63.0, 4.0, 7.0, 36.0, 4.0, 10.0, 8.0, 9.0, 3.0, 42.0, 4.0, 29.0, 12.0, 14.0, 8.0, 11.0, 3.0, 71.0, 18.0, 11.0, 117.0, 32.0, 52.0, 33.0, 97.0, 11.0, 8.0, 4.0, 4.0, 12.0, 74.0, 36.0, 6.0, 10.0, 27.0, 258.0, 24.0, 12.0, 212.0, 34.0, 9.0, 36.0, 22.0, 38.0, 42.0, 94.0, 65.0, 11.0, 21.0, 12.0, 39.0, 58.0, 648.0, 57.0, 7.0, 92.0, 31.0, 5.0, 2.0, 7.0, 321.0, 100.0, 178.0, 17.0, 81.0, 44.0, 8.0, 60.0, 17.0, 65.0, 20.0, 3.0, 71.0, 35.0, 31.0, 45.0, 34.0, 6.0, 7.0, 10.0, 173.0, 20.0, 338.0, 194.0, 108.0, 60.0, 88.0, 128.0, 48.0, 45.0, 175.0, 83.0, 59.0, 10.0, 285.0, 247.0, 44.0, 55.0, 86.0, 41.0, 143.0, 40.0, 107.0, 319.0, 141.0, 25.0, 32.0, 75.0, 54.0, 25.0, 27.0, 74.0, 80.0, 225.0, 11.0, 68.0, 131.0, 126.0, 209.0, 30.0, 62.0, 57.0, 8.0, 17.0, 50.0, 8.0, 9.0, 18.0, 12.0, 269.0, 306.0, 362.0, 56.0, 236.0, 90.0, 164.0, 603.0, 150.0, 1332.0, 123.0, 114.0, 1022.0, 949.0, 275.0, 69.0, 373.0, 307.0, 227.0, 398.0, 301.0, 170.0, 435.0, 293.0, 201.0, 529.0, 85.0, 94.0, 272.0, 175.0, 494.0, 160.0, 120.0, 143.0, 96.0, 142.0, 121.0, 41.0, 146.0, 250.0, 104.0, 88.0, 307.0, 194.0, 82.0, 152.0, 143.0, 251.0, 83.0, 14.0, 37.0, 41.0, 252.0, 8.0, 30.0, 134.0, 21.0, 70.0, 78.0, 48.0, 80.0, 158.0, 153.0, 127.0, 26.0, 180.0, 142.0, 51.0, 56.0, 174.0, 75.0, 34.0, 218.0, 214.0, 105.0, 72.0, 217.0, 598.0, 45.0, 169.0, 96.0, 177.0, 207.0, 104.0, 19.0, 42.0, 24.0, 205.0, 15.0, 164.0, 385.0, 161.0, 60.0, 86.0, 60.0, 22.0, 131.0, 265.0, 61.0, 142.0, 187.0, 154.0, 14.0, 337.0, 387.0, 303.0, 63.0, 75.0, 7.0, 30.0, 185.0, 233.0, 133.0, 289.0, 88.0, 10.0, 40.0, 344.0, 164.0, 108.0, 372.0, 156.0, 204.0, 159.0, 84.0, 66.0, 226.0, 24.0, 228.0, 229.0, 152.0, 127.0, 114.0, 156.0, 46.0, 147.0, 139.0, 12.0, 108.0, 322.0, 16.0, 69.0, 197.0, 74.0, 25.0, 33.0, 19.0, 68.0, 134.0, 106.0, 59.0, 100.0, 55.0, 18.0, 6.0, 48.0, 28.0, 339.0, 9.0, 77.0, 40.0, 14.0, 100.0, 29.0, 142.0, 26.0, 5.0, 494.0, 3.0, 38.0, 129.0, 106.0, 31.0, 5.0, 85.0, 91.0, 103.0, 3.0, 27.0, 82.0, 393.0, 5.0, 68.0, 198.0, 124.0, 6.0, 46.0, 112.0, 73.0, 179.0, 385.0, 32.0, 46.0, 292.0, 152.0, 2.0, 3.0, 50.0, 156.0, 3.0, 95.0, 108.0, 28.0, 156.0, 131.0, 7.0, 339.0, 7.0, 157.0, 336.0, 6.0, 67.0, 129.0, 109.0, 43.0, 176.0, 97.0, 2.0, 5.0, 113.0, 12.0, 24.0, 118.0, 75.0, 123.0, 61.0, 12.0, 112.0, 316.0, 159.0, 105.0, 223.0, 3.0, 90.0, 3.0, 5.0, 54.0, 88.0, 4.0, 73.0, 8.0, 73.0, 25.0, 111.0, 22.0, 16.0, 20.0, 3.0, 128.0, 26.0, 159.0, 62.0, 291.0, 41.0, 32.0, 82.0, 147.0, 36.0, 22.0, 98.0, 75.0, 111.0, 40.0, 144.0, 64.0, 128.0, 47.0, 6.0, 170.0, 83.0, 26.0, 7.0, 19.0, 27.0, 40.0, 528.0, 11.0, 56.0, 32.0, 263.0, 12.0, 119.0, 53.0, 84.0, 34.0, 114.0, 41.0, 247.0, 230.0, 221.0, 113.0, 17.0, 32.0, 117.0, 24.0, 36.0, 392.0, 17.0, 17.0, 5.0, 158.0, 162.0, 317.0, 44.0, 9.0, 160.0, 333.0, 3.0, 27.0, 61.0, 23.0, 30.0, 269.0, 313.0, 478.0, 268.0, 359.0, 142.0, 26.0, 47.0, 225.0, 163.0, 321.0, 93.0, 42.0, 39.0, 69.0, 56.0, 49.0, 49.0, 111.0, 290.0, 284.0, 22.0, 392.0, 502.0, 380.0, 549.0, 374.0, 97.0, 223.0, 99.0, 106.0, 189.0, 143.0, 75.0, 83.0, 126.0, 174.0, 215.0, 262.0, 99.0, 1585.0, 314.0, 79.0, 292.0, 259.0, 400.0, 334.0, 368.0, 108.0, 75.0, 400.0, 363.0, 130.0, 406.0, 21.0, 498.0, 164.0, 261.0, 322.0, 212.0, 61.0, 411.0, 308.0, 199.0, 65.0, 83.0, 65.0, 351.0, 94.0, 777.0, 213.0, 272.0, 302.0, 157.0, 425.0, 51.0, 64.0, 218.0, 225.0, 331.0, 211.0, 111.0, 217.0, 116.0, 77.0, 361.0, 179.0, 86.0, 43.0, 1862.0, 1708.0, 14500.0, 3698.0, 6212.0, 7332.0, 4344.0, 5090.0, 3751.0, 3567.0, 2739.0, 3737.0, 5468.0, 2952.0, 2568.0, 2353.0, 5078.0, 3358.0, 1532.0, 3029.0, 3973.0, 3498.0, 4139.0, 1613.0, 1194.0, 2016.0, 1867.0, 2260.0, 1780.0, 2467.0, 3106.0, 1880.0, 2983.0, 5683.0, 3415.0, 2531.0, 2777.0, 2861.0, 2591.0, 2484.0, 3313.0, 802.0, 1707.0, 1778.0, 1547.0, 2047.0, 1524.0, 1564.0, 959.0, 864.0, 2128.0, 2449.0, 1138.0, 1212.0, 1726.0, 1355.0, 1028.0, 3090.0, 2949.0, 1934.0, 2140.0, 1157.0, 2593.0, 1044.0, 530.0, 1911.0, 1653.0, 362.0, 2602.0, 1009.0, 1956.0, 1697.0, 1771.0, 1336.0, 1263.0, 2577.0, 1837.0, 3448.0, 844.0, 1739.0, 2635.0, 3043.0, 5253.0, 2347.0, 1745.0, 1582.0, 2980.0, 1700.0, 1496.0, 986.0, 2623.0, 4047.0, 3409.0, 1494.0, 3562.0, 6755.0, 1488.0, 1819.0, 1607.0, 918.0, 1581.0, 3083.0, 1305.0, 1191.0, 2064.0, 1406.0, 1406.0, 2459.0, 2065.0, 3528.0, 3036.0, 1099.0, 351.0, 1481.0, 399.0, 1834.0, 1885.0, 1761.0, 887.0, 754.0, 1753.0, 978.0, 1691.0, 705.0, 1123.0, 1621.0, 1005.0, 1482.0, 1851.0, 1260.0, 822.0, 930.0, 375.0, 1040.0, 1426.0, 803.0, 503.0, 1491.0, 774.0, 1633.0, 1065.0, 1539.0, 924.0, 2624.0, 4999.0, 3099.0, 473.0, 1471.0, 1181.0, 759.0, 386.0, 2157.0, 1972.0, 1052.0, 2163.0, 1134.0, 899.0, 587.0, 1919.0, 1723.0, 1119.0, 4335.0, 1755.0, 6740.0, 2117.0, 2446.0, 1484.0, 3047.0, 2138.0, 2155.0, 1750.0, 2818.0, 2469.0, 2294.0, 1699.0, 325.0, 4767.0, 3486.0, 1777.0, 5129.0, 1093.0, 1136.0, 1107.0, 1259.0, 1676.0, 1117.0, 1232.0, 915.0, 1116.0, 1770.0, 1334.0, 1279.0, 980.0, 1047.0, 1175.0, 1538.0, 1091.0, 502.0, 551.0, 1825.0, 742.0, 2097.0, 1819.0, 376.0, 1675.0, 770.0, 568.0, 715.0, 766.0, 1194.0, 784.0, 1961.0, 892.0, 579.0, 766.0, 1219.0, 81.0, 586.0, 1936.0, 807.0, 631.0, 766.0, 163.0, 1876.0, 1940.0, 849.0, 607.0, 1981.0, 884.0, 1215.0, 1750.0, 565.0, 1575.0, 390.0, 2033.0, 526.0, 993.0, 533.0, 521.0, 192.0, 974.0, 846.0, 636.0, 1537.0, 1643.0, 2568.0, 380.0, 1332.0, 1117.0, 930.0, 1096.0, 514.0, 62.0, 1189.0, 76.0, 603.0, 1273.0, 339.0, 1189.0, 424.0, 1324.0, 907.0, 531.0, 2410.0, 1770.0, 268.0, 1262.0, 1133.0, 1269.0, 2673.0, 2314.0, 2503.0, 5742.0, 1780.0, 4426.0, 3546.0, 2350.0, 1693.0, 180.0, 1883.0, 943.0, 1911.0, 723.0, 768.0, 926.0, 1035.0, 1265.0, 663.0, 221.0, 973.0, 2212.0, 553.0, 1214.0, 3535.0, 1550.0, 915.0, 2124.0, 2050.0, 1631.0, 2696.0, 880.0, 1342.0, 1471.0, 967.0, 745.0, 2597.0, 3541.0, 2042.0, 2070.0, 1357.0, 1215.0, 976.0, 2071.0, 1999.0, 1286.0, 1505.0, 1222.0, 1097.0, 1944.0, 2062.0, 775.0, 4217.0, 4412.0, 4266.0, 570.0, 2397.0, 1322.0, 1034.0, 740.0, 459.0, 745.0, 477.0, 1209.0, 1521.0, 800.0, 839.0, 807.0, 1161.0, 2079.0, 1261.0, 1059.0, 489.0, 470.0, 528.0, 1296.0, 936.0, 1086.0, 1471.0, 632.0, 477.0, 876.0, 1176.0, 1032.0, 2076.0, 1025.0, 351.0, 1282.0, 1254.0, 1010.0, 1122.0, 555.0, 339.0, 1942.0, 763.0, 1344.0, 608.0, 1238.0, 2081.0, 1079.0, 1185.0, 767.0, 1180.0, 1309.0, 1541.0, 1759.0, 2111.0, 137.0, 811.0, 2105.0, 1166.0, 3679.0, 156.0, 3430.0, 2178.0, 2035.0, 2698.0, 2714.0, 4192.0, 5704.0, 2110.0, 5273.0, 45705.0, 22964.0, 2606.0, 353.0, 2390.0, 1642.0, 1202.0, 976.0, 873.0, 1044.0, 1299.0, 1811.0, 1756.0, 566.0, 3925.0, 7060.0]})}, {'time': ('2016-12-29 16:31:00', {'close': [2245.25, 2245.25, 2245.25, 2245.5, 2245.25, 2245.25, 2245.5, 2246.0, 2246.0, 2246.0, 2245.75, 2246.0, 2246.0, 2246.25, 2246.25, 2246.25, 2246.25, 2246.5, 2246.25, 2246.25, 2246.25, 2246.25, 2246.5, 2246.5, 2246.25, 2246.5, 2246.25, 2246.0, 2246.0, 2246.25], 'volume': [659.0, 31.0, 13.0, 95.0, 124.0, 24.0, 181.0, 863.0, 364.0, 2.0, 8.0, 88.0, 25.0, 77.0, 322.0, 47.0, 304.0, 37.0, 106.0, 90.0, 27.0, 93.0, 3.0, 8.0, 44.0, 33.0, 127.0, 116.0, 161.0, 450.0]})}, {'time': ('2016-12-29 18:01:00', {'close': [2246.25, 2245.75, 2246.0, 2245.75, 2245.75, 2246.25, 2246.25, 2246.25, 2246.25, 2246.0, 2246.0, 2246.0, 2246.25, 2246.25, 2246.25, 2246.25, 2246.0, 2246.25, 2246.25, 2246.25, 2246.25, 2246.25, 2246.0, 2246.25, 2246.0, 2246.0, 2246.25, 2246.25, 2246.25, 2246.25, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.25, 2246.25, 2246.5, 2246.5, 2246.75, 2246.5, 2246.5, 2246.5, 2246.75, 2246.75, 2247.0, 2247.0, 2246.75, 2246.75, 2246.5, 2246.0, 2246.0, 2246.0, 2246.25, 2246.0, 2246.25, 2246.0, 2246.0, 2245.75, 2246.0, 2246.0, 2245.5, 2245.5, 2245.25, 2245.5, 2245.75, 2245.75, 2245.75, 2245.75, 2245.75, 2245.75, 2246.0, 2246.0, 2245.75, 2245.75, 2246.25, 2246.25, 2246.0, 2246.25, 2246.0, 2246.25, 2246.25, 2246.25, 2246.0, 2246.25, 2246.25, 2246.0, 2245.5, 2245.25, 2245.25, 2245.0, 2245.0, 2244.75, 2245.0, 2245.0, 2245.25, 2245.5, 2245.5, 2245.5, 2245.5, 2245.5, 2245.5, 2245.75, 2246.0, 2245.75, 2245.5, 2245.75, 2245.5, 2245.75, 2245.5, 2245.75, 2245.5, 2245.5, 2245.5, 2245.5, 2245.75, 2245.75, 2245.5, 2245.75, 2245.25, 2245.5, 2245.75, 2245.75, 2245.5, 2245.75, 2246.0, 2246.0, 2246.25, 2246.25, 2246.5, 2246.25, 2246.25, 2246.25, 2246.75, 2246.0, 2245.5, 2245.5, 2245.5, 2245.25, 2245.75, 2245.5, 2245.75, 2245.75, 2245.5, 2245.75, 2245.5, 2245.75, 2246.0, 2246.0, 2246.0, 2246.0, 2245.75, 2246.0, 2246.5, 2246.25, 2246.5, 2246.5, 2246.25, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.5, 2246.75, 2246.75, 2247.0, 2247.0, 2247.0, 2247.0, 2246.75, 2246.75, 2246.5, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.75, 2246.5, 2246.25, 2246.25, 2246.5, 2246.25, 2246.25, 2246.25, 2246.25, 2246.25, 2246.25, 2246.5, 2246.5, 2246.25, 2246.25, 2246.25, 2246.5, 2246.75, 2246.5, 2246.75, 2246.5, 2246.75, 2247.0, 2246.75, 2247.0, 2247.0, 2247.0, 2247.25, 2247.25, 2247.25, 2247.25, 2247.25, 2247.25, 2247.5, 2247.0, 2247.25, 2247.25, 2247.25, 2247.25, 2247.25, 2247.0, 2247.25, 2247.75, 2247.5, 2247.25, 2247.25, 2247.25, 2247.25, 2247.25, 2247.25, 2247.0, 2247.25, 2247.0, 2247.0, 2247.25, 2247.0, 2247.25, 2247.25, 2247.25, 2247.0, 2247.0, 2246.75, 2247.0, 2246.75, 2246.75, 2246.75, 2246.5, 2246.75, 2246.5, 2247.0, 2247.0, 2247.0, 2247.0, 2247.25, 2247.25, 2247.0, 2247.0, 2247.25, 2246.75, 2246.75, 2247.0, 2247.75, 2247.75, 2247.75, 2247.75, 2247.75, 2247.5, 2247.75, 2247.75, 2247.5, 2247.75, 2247.5, 2247.5, 2248.0, 2248.25, 2248.5, 2248.5, 2248.5, 2248.5, 2248.5, 2248.25, 2248.25, 2248.25, 2248.75, 2248.75, 2248.75, 2248.75, 2248.75, 2248.5, 2248.75, 2248.75, 2248.75, 2248.5, 2248.75, 2248.75, 2248.75, 2249.0, 2249.0, 2248.75, 2249.0, 2249.0, 2248.75, 2248.5, 2249.0, 2249.0, 2249.25, 2249.25, 2249.25, 2249.25, 2249.25, 2249.5, 2249.5, 2249.5, 2249.75, 2249.75, 2249.5, 2249.5, 2249.25, 2249.5, 2249.75, 2249.25, 2249.75, 2249.5, 2249.75, 2249.75, 2249.75, 2249.75, 2249.75, 2250.0, 2249.75, 2250.0, 2249.75, 2249.75, 2250.0, 2249.75, 2249.75, 2249.75, 2249.75, 2250.0, 2249.75, 2249.75, 2250.0, 2249.75, 2249.75, 2249.5, 2249.75, 2249.5, 2249.5, 2249.5, 2249.75, 2249.75, 2249.5, 2249.5, 2249.5, 2249.75, 2250.0, 2250.25, 2250.25, 2250.75, 2251.75, 2251.5, 2251.5, 2251.25, 2251.5, 2251.75, 2251.25, 2251.5, 2251.25, 2251.5, 2251.25, 2251.5, 2251.25, 2251.25, 2251.25, 2251.5, 2251.5, 2251.25, 2251.75, 2252.0, 2252.0, 2252.0, 2252.25, 2252.25, 2252.0, 2252.25, 2252.25, 2251.75, 2251.5, 2251.75, 2251.5, 2251.75, 2251.5, 2251.5, 2251.75, 2251.5, 2252.0, 2251.75, 2252.0, 2252.0, 2251.75, 2251.75, 2252.0, 2251.75, 2251.5, 2251.75, 2251.5, 2251.25, 2251.5, 2251.25, 2251.25, 2251.25, 2251.25, 2251.25, 2251.5, 2251.5, 2251.5, 2251.75, 2251.5, 2251.75, 2251.25, 2251.25, 2251.25, 2251.0, 2251.0, 2251.25, 2251.25, 2251.25, 2251.25, 2251.25, 2251.25, 2251.25, 2251.25, 2251.0, 2251.0, 2251.0, 2251.25, 2251.0, 2251.0, 2251.0, 2251.25, 2251.25, 2251.25, 2251.0, 2251.25, 2251.25, 2251.25, 2251.5, 2252.0, 2251.75, 2251.5, 2251.5, 2251.75, 2251.5, 2251.75, 2251.5, 2251.5, 2251.5, 2251.5, 2251.5, 2251.5, 2251.5, 2251.5, 2251.25, 2251.5, 2251.0, 2251.0, 2251.0, 2251.0, 2251.0, 2251.0, 2250.75, 2250.75, 2250.5, 2250.5, 2250.5, 2250.25, 2250.25, 2250.25, 2250.0, 2250.25, 2250.25, 2250.25, 2250.25, 2250.25, 2250.0, 2250.0, 2250.25, 2250.25, 2250.5, 2250.5, 2250.25, 2250.5, 2250.5, 2250.25, 2250.25, 2250.25, 2250.25, 2250.25, 2250.25, 2250.25, 2250.25, 2250.25, 2250.25, 2250.0, 2250.0, 2249.75, 2250.25, 2250.0, 2250.25, 2250.25, 2250.5, 2250.75, 2250.5, 2250.5, 2251.0, 2250.75, 2250.5, 2250.5, 2250.5, 2250.5, 2250.75, 2250.5, 2250.5, 2250.25, 2250.0, 2250.0, 2250.0, 2250.0, 2250.5, 2250.25, 2250.0, 2250.25, 2250.25, 2250.25, 2250.5, 2250.5, 2249.75, 2249.75, 2249.5, 2249.5, 2249.0, 2249.0, 2249.25, 2248.75, 2248.75, 2248.75, 2248.5, 2248.5, 2248.5, 2248.25, 2248.25, 2248.75, 2249.0, 2248.75, 2248.75, 2248.75, 2248.75, 2248.5, 2248.5, 2249.0, 2249.25, 2249.25, 2248.5, 2248.75, 2248.75, 2248.5, 2248.75, 2249.0, 2248.75, 2248.75, 2248.75, 2248.75, 2249.0, 2249.0, 2248.75, 2249.0, 2248.75, 2248.75, 2248.75, 2249.0, 2248.75, 2249.25, 2248.75, 2249.0, 2248.75, 2249.0, 2248.75, 2249.25, 2249.25, 2249.25, 2249.25, 2249.25, 2249.5, 2249.5, 2249.75, 2249.75, 2249.75, 2249.75, 2249.5, 2249.75, 2249.75, 2250.0, 2250.0, 2250.0, 2250.0, 2249.75, 2249.75, 2249.75, 2249.5, 2249.75, 2250.0, 2249.5, 2249.75, 2249.5, 2249.75, 2250.0, 2249.75, 2249.75, 2249.75, 2250.0, 2249.75, 2250.0, 2250.0, 2249.75, 2249.75, 2250.0, 2249.5, 2249.5, 2249.75, 2249.5, 2249.5, 2249.25, 2249.25, 2249.75, 2249.5, 2249.25, 2249.5, 2249.5, 2249.5, 2249.75, 2249.75, 2249.75, 2250.0, 2250.0, 2250.0, 2250.0, 2250.25, 2250.0, 2250.25, 2250.0, 2250.25, 2250.5, 2250.25, 2250.25, 2250.5, 2250.0, 2249.75, 2249.75, 2250.0, 2250.0, 2250.0, 2250.0, 2249.75, 2249.75, 2250.0, 2249.75, 2249.75, 2250.0, 2250.0, 2250.0, 2250.0, 2250.25, 2250.0, 2250.25, 2250.25, 2250.25, 2250.25, 2249.75, 2249.5, 2249.25, 2249.25, 2249.75, 2249.75, 2249.75, 2249.5, 2249.75, 2249.75, 2249.75, 2249.75, 2249.75, 2249.75, 2249.75, 2249.5, 2249.5, 2249.75, 2249.75, 2250.0, 2250.25, 2250.25, 2250.25, 2250.0, 2250.0, 2250.0, 2250.75, 2250.5, 2250.75, 2250.5, 2250.5, 2250.25, 2250.25, 2250.0, 2250.25, 2250.0, 2249.75, 2250.0, 2250.0, 2249.75, 2249.75, 2249.75, 2249.5, 2249.5, 2249.5, 2249.25, 2249.25, 2249.25, 2249.0, 2249.25, 2249.0, 2249.25, 2249.0, 2249.0, 2249.0, 2248.75, 2249.5, 2249.75, 2249.75, 2249.75, 2249.75, 2250.0, 2249.75, 2250.25, 2250.0, 2249.75, 2249.5, 2249.5, 2249.75, 2249.5, 2249.25, 2249.0, 2249.25, 2249.25, 2249.25, 2249.75, 2249.5, 2249.5, 2249.5, 2249.5, 2249.5, 2249.25, 2249.75, 2249.5, 2249.75, 2249.75, 2249.75, 2250.0, 2249.75, 2250.0, 2250.0, 2250.0, 2250.0, 2250.0, 2249.75, 2249.5, 2249.25, 2249.0, 2249.0, 2248.75, 2248.75, 2249.0, 2249.0, 2249.0, 2248.75, 2248.5, 2248.75, 2248.75, 2248.25, 2248.75, 2248.5, 2248.0, 2248.5, 2248.25, 2249.0, 2249.0, 2249.25, 2249.25, 2249.5, 2249.5, 2249.0, 2249.25, 2249.0, 2249.75, 2249.5, 2249.25, 2249.75, 2249.5, 2249.75, 2249.75, 2250.0, 2250.0, 2250.5, 2250.5, 2250.75, 2250.75, 2250.75, 2250.5, 2250.25, 2250.25, 2250.5, 2250.5, 2250.25, 2250.5, 2250.25, 2250.25, 2250.25, 2250.5, 2250.75, 2250.75, 2250.75, 2251.0, 2251.25, 2251.25, 2251.0, 2250.75, 2250.75, 2250.75, 2251.5, 2251.25, 2251.0, 2251.25, 2251.0, 2251.0, 2250.75, 2250.75, 2250.5, 2250.75, 2250.75, 2250.75, 2250.5, 2250.5, 2250.5, 2250.5, 2250.25, 2250.25, 2250.5, 2250.0, 2250.5, 2250.75, 2251.0, 2250.75, 2250.75, 2251.0, 2250.75, 2250.75, 2251.0, 2251.25, 2251.25, 2251.0, 2251.5, 2251.25, 2251.25, 2251.0, 2251.25, 2251.25, 2251.0, 2250.75, 2250.75, 2251.0, 2250.75, 2250.75, 2250.5, 2250.25, 2250.25, 2250.25, 2250.0, 2250.0, 2250.0, 2250.0, 2250.0, 2250.0, 2250.25, 2249.5, 2249.75, 2249.5, 2249.5, 2250.0, 2250.25, 2250.0, 2249.75, 2250.0, 2250.0, 2249.5, 2249.5, 2249.25, 2249.75, 2250.0, 2249.75, 2249.25, 2249.5, 2249.5, 2249.25, 2249.5, 2249.75, 2249.75, 2249.5, 2249.5, 2249.75, 2249.75, 2249.5, 2249.0, 2248.75, 2248.5, 2247.5, 2248.5, 2247.75, 2246.25, 2244.25, 2246.0, 2244.25, 2244.75, 2242.5, 2242.5, 2243.0, 2243.75, 2243.5, 2241.75, 2241.5, 2242.25, 2243.25, 2243.5, 2243.25, 2242.75, 2242.0, 2242.0, 2242.25, 2242.0, 2241.75, 2241.0, 2240.0, 2238.75, 2240.5, 2238.75, 2239.0, 2238.5, 2238.5, 2238.75, 2238.0, 2239.25, 2239.5, 2239.0, 2239.0, 2239.0, 2239.5, 2240.25, 2239.75, 2239.5, 2240.0, 2239.25, 2240.0, 2239.25, 2239.25, 2238.25, 2238.5, 2239.0, 2237.75, 2237.0, 2237.5, 2237.25, 2237.25, 2237.25, 2237.75, 2237.25, 2238.5, 2238.75, 2239.25, 2239.5, 2239.5, 2239.25, 2238.75, 2239.0, 2239.0, 2238.75, 2239.0, 2238.75, 2238.5, 2238.75, 2238.25, 2238.25, 2237.5, 2238.75, 2238.75, 2238.5, 2239.25, 2238.5, 2238.75, 2239.0, 2238.75, 2239.5, 2239.25, 2239.75, 2239.75, 2240.0, 2239.5, 2239.75, 2240.0, 2239.75, 2240.0, 2239.5, 2239.25, 2238.75, 2238.75, 2238.5, 2238.75, 2238.75, 2238.5, 2238.75, 2238.75, 2239.25, 2239.25, 2239.75, 2239.25, 2239.5, 2239.25, 2239.75, 2240.0, 2240.5, 2240.5, 2241.25, 2241.5, 2241.5, 2241.0, 2240.75, 2241.0, 2241.25, 2241.5, 2241.0, 2241.75, 2241.0, 2241.0, 2240.75, 2240.5, 2240.5, 2240.5, 2240.75, 2242.0, 2241.75, 2242.0, 2242.5, 2242.5, 2242.0, 2242.0, 2241.75, 2242.25, 2242.5, 2242.0, 2242.5, 2241.75, 2241.75, 2241.5, 2241.25, 2241.25, 2240.75, 2240.25, 2240.0, 2239.0, 2238.75, 2239.0, 2238.75, 2239.25, 2239.25, 2239.0, 2239.75, 2239.25, 2240.0, 2240.5, 2240.5, 2240.5, 2240.25, 2239.5, 2239.5, 2239.75, 2239.75, 2240.0, 2240.25, 2240.25, 2240.0, 2239.75, 2239.5, 2239.25, 2239.75, 2239.5, 2239.0, 2239.25, 2239.75, 2239.5, 2239.5, 2239.25, 2238.25, 2237.75, 2238.5, 2238.0, 2237.75, 2237.75, 2237.75, 2237.5, 2237.5, 2237.75, 2237.25, 2238.0, 2237.75, 2237.75, 2237.5, 2235.75, 2235.0, 2235.75, 2235.75, 2235.75, 2235.5, 2236.0, 2235.75, 2236.25, 2236.5, 2236.5, 2236.75, 2236.5, 2236.5, 2236.5, 2236.25, 2236.5, 2236.25, 2236.0, 2236.0, 2236.5, 2236.25, 2236.5, 2236.5, 2236.5, 2236.5, 2237.25, 2237.25, 2237.25, 2238.0, 2237.5, 2237.75, 2238.0, 2238.5, 2238.5, 2238.5, 2237.75, 2238.0, 2238.0, 2237.75, 2238.25, 2237.75, 2237.5, 2237.5, 2238.0, 2238.5, 2238.0, 2238.0, 2238.0, 2237.5, 2237.75, 2237.75, 2238.0, 2237.5, 2236.25, 2236.25, 2236.5, 2236.5, 2236.0, 2236.0, 2235.5, 2235.75, 2236.5, 2236.75, 2236.5, 2236.0, 2235.5, 2235.5, 2236.0, 2235.25, 2235.5, 2235.75, 2235.25, 2235.0, 2235.0, 2235.0, 2235.25, 2235.0, 2234.75, 2234.5, 2234.5, 2233.5, 2233.75, 2233.75, 2234.0, 2233.75, 2234.5, 2234.5, 2234.5, 2234.5, 2234.5, 2234.0, 2233.75, 2233.75, 2233.5, 2234.0, 2233.25, 2233.25, 2233.5, 2233.0, 2232.0, 2232.75, 2233.25, 2233.25, 2233.25, 2233.25, 2233.5, 2233.75, 2234.25, 2234.25, 2234.25, 2233.25, 2233.75, 2233.75, 2233.75, 2233.0, 2232.25, 2231.75, 2231.0, 2230.25, 2229.0, 2230.5, 2230.0, 2230.0, 2229.25, 2228.5, 2228.5, 2229.0, 2229.25, 2229.75, 2229.75, 2229.0, 2229.5, 2229.75, 2230.25, 2230.0, 2230.5, 2231.0, 2230.5, 2231.0, 2230.5, 2231.0, 2231.25, 2232.0, 2232.5, 2233.5, 2233.0, 2233.0, 2232.75, 2232.75, 2232.5, 2233.0, 2233.5, 2233.5, 2232.75, 2233.25, 2233.5, 2234.0, 2233.75, 2234.5, 2234.5, 2233.75, 2233.25, 2233.75, 2233.5, 2233.5, 2232.75, 2232.5, 2233.0, 2232.5, 2232.75, 2232.0, 2231.0, 2231.0, 2230.75, 2231.25, 2231.25, 2231.0, 2230.75, 2230.25, 2229.75, 2229.5, 2229.5, 2230.0, 2230.25, 2232.0, 2232.0, 2234.25, 2235.0, 2236.75, 2235.0, 2234.0, 2235.0, 2234.75, 2235.0, 2235.25, 2235.0, 2235.5, 2235.75, 2235.5, 2235.25, 2235.25, 2235.75, 2235.25, 2234.75], 'volume': [480.0, 291.0, 60.0, 52.0, 11.0, 80.0, 9.0, 7.0, 7.0, 76.0, 62.0, 28.0, 5.0, 56.0, 18.0, 4.0, 52.0, 3.0, 5.0, 17.0, 94.0, 16.0, 8.0, 3.0, 11.0, 11.0, 15.0, 3.0, 5.0, 11.0, 78.0, 26.0, 14.0, 59.0, 32.0, 10.0, 93.0, 13.0, 45.0, 32.0, 88.0, 43.0, 58.0, 20.0, 28.0, 125.0, 149.0, 43.0, 72.0, 36.0, 21.0, 20.0, 22.0, 121.0, 62.0, 12.0, 27.0, 137.0, 2.0, 119.0, 156.0, 99.0, 81.0, 58.0, 161.0, 129.0, 35.0, 19.0, 148.0, 43.0, 82.0, 34.0, 60.0, 33.0, 62.0, 23.0, 132.0, 49.0, 209.0, 67.0, 52.0, 41.0, 80.0, 57.0, 66.0, 62.0, 154.0, 5.0, 40.0, 35.0, 251.0, 112.0, 54.0, 116.0, 18.0, 209.0, 101.0, 23.0, 110.0, 87.0, 31.0, 14.0, 13.0, 16.0, 3.0, 156.0, 109.0, 41.0, 93.0, 53.0, 9.0, 6.0, 85.0, 10.0, 41.0, 16.0, 5.0, 46.0, 11.0, 3.0, 76.0, 38.0, 31.0, 29.0, 38.0, 39.0, 59.0, 50.0, 20.0, 87.0, 11.0, 17.0, 63.0, 4.0, 171.0, 13.0, 95.0, 321.0, 209.0, 122.0, 40.0, 35.0, 30.0, 13.0, 10.0, 32.0, 18.0, 24.0, 29.0, 19.0, 64.0, 204.0, 21.0, 14.0, 58.0, 52.0, 78.0, 28.0, 83.0, 18.0, 34.0, 54.0, 44.0, 38.0, 9.0, 49.0, 23.0, 224.0, 74.0, 19.0, 9.0, 27.0, 25.0, 53.0, 46.0, 5.0, 120.0, 7.0, 180.0, 25.0, 15.0, 73.0, 132.0, 26.0, 26.0, 51.0, 6.0, 170.0, 51.0, 6.0, 32.0, 105.0, 14.0, 13.0, 69.0, 61.0, 15.0, 0.0, 13.0, 3.0, 5.0, 4.0, 3.0, 15.0, 3.0, 13.0, 28.0, 8.0, 33.0, 18.0, 37.0, 45.0, 16.0, 8.0, 7.0, 107.0, 32.0, 31.0, 25.0, 83.0, 15.0, 151.0, 29.0, 37.0, 45.0, 27.0, 14.0, 7.0, 10.0, 10.0, 21.0, 231.0, 74.0, 75.0, 65.0, 30.0, 9.0, 3.0, 8.0, 6.0, 6.0, 81.0, 15.0, 6.0, 14.0, 11.0, 9.0, 10.0, 12.0, 20.0, 73.0, 22.0, 121.0, 18.0, 17.0, 69.0, 10.0, 20.0, 69.0, 70.0, 10.0, 5.0, 17.0, 87.0, 18.0, 8.0, 8.0, 251.0, 13.0, 13.0, 228.0, 133.0, 16.0, 12.0, 16.0, 5.0, 9.0, 9.0, 20.0, 8.0, 11.0, 78.0, 249.0, 78.0, 358.0, 112.0, 13.0, 43.0, 6.0, 11.0, 1.0, 17.0, 75.0, 34.0, 12.0, 21.0, 125.0, 64.0, 44.0, 18.0, 27.0, 32.0, 13.0, 189.0, 83.0, 63.0, 121.0, 10.0, 118.0, 138.0, 24.0, 66.0, 236.0, 43.0, 33.0, 29.0, 4.0, 25.0, 44.0, 322.0, 252.0, 204.0, 16.0, 60.0, 42.0, 41.0, 59.0, 9.0, 285.0, 79.0, 91.0, 35.0, 185.0, 117.0, 99.0, 73.0, 16.0, 32.0, 110.0, 92.0, 65.0, 64.0, 12.0, 94.0, 45.0, 13.0, 7.0, 11.0, 13.0, 54.0, 5.0, 37.0, 12.0, 51.0, 15.0, 11.0, 34.0, 11.0, 15.0, 7.0, 207.0, 134.0, 99.0, 121.0, 76.0, 429.0, 187.0, 126.0, 1122.0, 258.0, 65.0, 85.0, 58.0, 28.0, 67.0, 24.0, 7.0, 8.0, 4.0, 15.0, 22.0, 11.0, 26.0, 42.0, 40.0, 13.0, 693.0, 134.0, 36.0, 617.0, 767.0, 143.0, 17.0, 23.0, 62.0, 295.0, 28.0, 14.0, 23.0, 31.0, 13.0, 16.0, 24.0, 155.0, 288.0, 199.0, 40.0, 10.0, 1.0, 126.0, 28.0, 25.0, 35.0, 25.0, 67.0, 5.0, 78.0, 47.0, 80.0, 36.0, 7.0, 92.0, 10.0, 247.0, 51.0, 21.0, 10.0, 53.0, 106.0, 5.0, 57.0, 16.0, 45.0, 112.0, 64.0, 70.0, 25.0, 7.0, 10.0, 0.0, 5.0, 2.0, 0.0, 4.0, 7.0, 4.0, 9.0, 2.0, 1.0, 106.0, 46.0, 10.0, 3.0, 56.0, 14.0, 14.0, 369.0, 192.0, 12.0, 60.0, 10.0, 123.0, 7.0, 6.0, 14.0, 0.0, 14.0, 7.0, 69.0, 53.0, 46.0, 139.0, 85.0, 35.0, 19.0, 17.0, 16.0, 16.0, 2.0, 137.0, 159.0, 74.0, 44.0, 20.0, 61.0, 35.0, 51.0, 10.0, 8.0, 79.0, 46.0, 33.0, 76.0, 2.0, 32.0, 87.0, 93.0, 109.0, 55.0, 23.0, 10.0, 9.0, 45.0, 53.0, 23.0, 24.0, 26.0, 4.0, 26.0, 14.0, 14.0, 33.0, 32.0, 56.0, 98.0, 65.0, 11.0, 76.0, 17.0, 77.0, 41.0, 2.0, 13.0, 79.0, 72.0, 126.0, 3.0, 27.0, 37.0, 72.0, 162.0, 3.0, 11.0, 66.0, 75.0, 48.0, 53.0, 22.0, 37.0, 87.0, 2735.0, 196.0, 72.0, 89.0, 81.0, 270.0, 269.0, 220.0, 372.0, 148.0, 306.0, 5.0, 98.0, 275.0, 82.0, 228.0, 158.0, 10.0, 211.0, 50.0, 466.0, 24.0, 15.0, 44.0, 11.0, 121.0, 34.0, 20.0, 224.0, 129.0, 68.0, 276.0, 120.0, 81.0, 22.0, 67.0, 8.0, 32.0, 12.0, 91.0, 151.0, 18.0, 70.0, 24.0, 5.0, 120.0, 23.0, 10.0, 15.0, 20.0, 238.0, 149.0, 10.0, 26.0, 10.0, 16.0, 64.0, 116.0, 62.0, 27.0, 426.0, 112.0, 43.0, 40.0, 65.0, 90.0, 45.0, 36.0, 18.0, 4.0, 55.0, 14.0, 4.0, 14.0, 43.0, 158.0, 79.0, 37.0, 222.0, 19.0, 121.0, 30.0, 35.0, 30.0, 155.0, 93.0, 68.0, 0.0, 2.0, 16.0, 315.0, 36.0, 53.0, 11.0, 3.0, 121.0, 1.0, 250.0, 3.0, 40.0, 157.0, 201.0, 250.0, 249.0, 52.0, 15.0, 39.0, 9.0, 158.0, 8.0, 65.0, 6.0, 53.0, 73.0, 177.0, 30.0, 7.0, 29.0, 62.0, 79.0, 460.0, 39.0, 10.0, 90.0, 111.0, 82.0, 39.0, 169.0, 17.0, 196.0, 148.0, 70.0, 208.0, 3.0, 7.0, 15.0, 121.0, 33.0, 33.0, 61.0, 12.0, 8.0, 79.0, 22.0, 327.0, 24.0, 311.0, 31.0, 112.0, 133.0, 91.0, 49.0, 4.0, 15.0, 39.0, 34.0, 147.0, 3.0, 2.0, 6.0, 159.0, 20.0, 123.0, 26.0, 119.0, 1.0, 82.0, 22.0, 59.0, 192.0, 100.0, 121.0, 406.0, 191.0, 16.0, 220.0, 10.0, 240.0, 105.0, 101.0, 82.0, 465.0, 33.0, 102.0, 178.0, 133.0, 133.0, 29.0, 51.0, 6.0, 9.0, 200.0, 6.0, 28.0, 616.0, 49.0, 30.0, 69.0, 112.0, 49.0, 40.0, 10.0, 328.0, 186.0, 121.0, 128.0, 5.0, 8.0, 5.0, 302.0, 101.0, 81.0, 136.0, 41.0, 4.0, 141.0, 84.0, 123.0, 61.0, 9.0, 28.0, 50.0, 6.0, 13.0, 9.0, 45.0, 24.0, 27.0, 124.0, 19.0, 10.0, 30.0, 33.0, 148.0, 121.0, 153.0, 10.0, 39.0, 20.0, 72.0, 168.0, 97.0, 200.0, 115.0, 41.0, 77.0, 52.0, 149.0, 15.0, 166.0, 10.0, 587.0, 46.0, 112.0, 226.0, 276.0, 162.0, 490.0, 203.0, 27.0, 254.0, 809.0, 94.0, 347.0, 118.0, 156.0, 97.0, 81.0, 63.0, 215.0, 106.0, 8.0, 167.0, 81.0, 1.0, 211.0, 90.0, 118.0, 53.0, 311.0, 280.0, 83.0, 14.0, 263.0, 54.0, 140.0, 176.0, 26.0, 15.0, 3.0, 15.0, 31.0, 397.0, 195.0, 144.0, 92.0, 67.0, 192.0, 308.0, 733.0, 272.0, 176.0, 190.0, 133.0, 344.0, 96.0, 122.0, 16.0, 128.0, 179.0, 35.0, 56.0, 90.0, 89.0, 795.0, 124.0, 58.0, 205.0, 21.0, 130.0, 126.0, 15.0, 68.0, 696.0, 219.0, 180.0, 109.0, 222.0, 128.0, 101.0, 112.0, 251.0, 50.0, 599.0, 132.0, 54.0, 467.0, 334.0, 101.0, 56.0, 287.0, 8.0, 526.0, 109.0, 42.0, 125.0, 470.0, 459.0, 189.0, 335.0, 951.0, 736.0, 482.0, 177.0, 80.0, 189.0, 66.0, 32.0, 237.0, 914.0, 461.0, 490.0, 259.0, 313.0, 1148.0, 635.0, 233.0, 15.0, 31.0, 218.0, 258.0, 154.0, 398.0, 355.0, 233.0, 532.0, 72.0, 63.0, 85.0, 39.0, 224.0, 369.0, 1033.0, 338.0, 130.0, 761.0, 521.0, 487.0, 526.0, 1298.0, 12365.0, 11210.0, 4372.0, 13978.0, 18944.0, 7932.0, 9357.0, 6204.0, 8843.0, 5970.0, 3865.0, 3353.0, 5582.0, 4586.0, 3389.0, 6174.0, 4408.0, 2498.0, 3652.0, 2343.0, 3371.0, 6006.0, 4530.0, 1292.0, 2255.0, 1927.0, 8227.0, 11240.0, 6795.0, 4284.0, 7855.0, 3563.0, 3125.0, 2189.0, 3333.0, 3287.0, 2559.0, 2985.0, 2165.0, 1381.0, 3021.0, 2967.0, 2337.0, 1790.0, 2495.0, 1956.0, 1984.0, 930.0, 771.0, 2052.0, 3185.0, 1751.0, 2717.0, 4806.0, 2523.0, 3465.0, 2213.0, 2182.0, 1779.0, 1036.0, 3680.0, 2753.0, 3175.0, 1982.0, 1444.0, 1096.0, 1526.0, 2067.0, 2167.0, 535.0, 1978.0, 2308.0, 943.0, 1341.0, 1842.0, 2244.0, 791.0, 2159.0, 811.0, 1273.0, 1708.0, 2255.0, 1968.0, 539.0, 600.0, 3502.0, 1533.0, 1481.0, 1775.0, 2141.0, 1124.0, 1783.0, 1589.0, 1919.0, 531.0, 618.0, 716.0, 1228.0, 433.0, 1659.0, 1456.0, 743.0, 1134.0, 1438.0, 1369.0, 1837.0, 870.0, 654.0, 1006.0, 571.0, 1285.0, 1559.0, 995.0, 1944.0, 1987.0, 2970.0, 3502.0, 2423.0, 1342.0, 2523.0, 2136.0, 905.0, 845.0, 955.0, 2212.0, 1376.0, 753.0, 674.0, 920.0, 553.0, 426.0, 310.0, 1979.0, 863.0, 3540.0, 1220.0, 2113.0, 1533.0, 1492.0, 690.0, 481.0, 838.0, 349.0, 1477.0, 1430.0, 890.0, 952.0, 309.0, 1057.0, 817.0, 4501.0, 1737.0, 1068.0, 2145.0, 1051.0, 2049.0, 1834.0, 393.0, 1113.0, 928.0, 1324.0, 1707.0, 1922.0, 1223.0, 740.0, 860.0, 713.0, 1174.0, 1076.0, 939.0, 396.0, 857.0, 791.0, 628.0, 494.0, 303.0, 846.0, 980.0, 588.0, 2102.0, 1091.0, 1204.0, 1842.0, 890.0, 361.0, 1696.0, 1366.0, 1311.0, 621.0, 1945.0, 1546.0, 2837.0, 2120.0, 1160.0, 1547.0, 2503.0, 1354.0, 991.0, 632.0, 971.0, 6506.0, 3938.0, 2372.0, 939.0, 1186.0, 888.0, 1047.0, 140.0, 943.0, 1384.0, 2181.0, 1891.0, 615.0, 1103.0, 581.0, 749.0, 452.0, 694.0, 1197.0, 402.0, 1361.0, 881.0, 729.0, 288.0, 97.0, 1051.0, 1949.0, 1594.0, 1836.0, 959.0, 1204.0, 435.0, 1333.0, 514.0, 1236.0, 1271.0, 764.0, 998.0, 1260.0, 1279.0, 752.0, 705.0, 875.0, 276.0, 519.0, 1646.0, 838.0, 951.0, 207.0, 835.0, 376.0, 1090.0, 358.0, 1007.0, 2391.0, 1090.0, 810.0, 815.0, 1932.0, 1557.0, 1652.0, 664.0, 1052.0, 1280.0, 1232.0, 419.0, 2181.0, 639.0, 418.0, 3803.0, 1211.0, 767.0, 2008.0, 1943.0, 1523.0, 1843.0, 156.0, 62.0, 1157.0, 1228.0, 580.0, 3846.0, 3957.0, 1916.0, 1078.0, 740.0, 1447.0, 1456.0, 423.0, 665.0, 126.0, 723.0, 292.0, 964.0, 139.0, 495.0, 892.0, 875.0, 1128.0, 1539.0, 2600.0, 2506.0, 2631.0, 1718.0, 1379.0, 411.0, 1164.0, 818.0, 1543.0, 1181.0, 2411.0, 1189.0, 1452.0, 592.0, 585.0, 3456.0, 5081.0, 3095.0, 6553.0, 2866.0, 7563.0, 4203.0, 1745.0, 960.0, 1377.0, 6702.0, 982.0, 1934.0, 1279.0, 2025.0, 1773.0, 2795.0, 2042.0, 2336.0, 2407.0, 1712.0, 1769.0, 1274.0, 1578.0, 683.0, 1391.0, 1556.0, 3552.0, 5671.0, 4535.0, 5044.0, 1670.0, 1795.0, 1756.0, 3438.0, 1405.0, 1638.0, 3560.0, 3458.0, 3648.0, 882.0, 733.0, 2604.0, 6207.0, 3089.0, 5495.0, 2202.0, 1343.0, 4737.0, 3320.0, 2857.0, 2085.0, 2892.0, 1325.0, 1766.0, 2496.0, 2081.0, 3412.0, 1553.0, 3418.0, 6193.0, 6554.0, 4950.0, 1820.0, 2489.0, 9200.0, 4974.0, 4090.0, 2812.0, 6621.0, 18544.0, 16924.0, 24471.0, 27059.0, 130873.0, 62422.0, 7213.0, 6467.0, 7872.0, 3511.0, 2984.0, 2238.0, 3134.0, 2167.0, 1159.0, 2627.0, 1470.0, 2604.0, 2836.0, 6140.0]})}, {'time': ('2016-12-30 16:31:00', {'close': [2235.0, 2235.25, 2235.0, 2234.5, 2234.0, 2234.0, 2234.25, 2234.0, 2233.75, 2233.5, 2233.5, 2233.25, 2233.75, 2233.75, 2233.75, 2234.0, 2234.25, 2234.0, 2233.75, 2234.25, 2234.25, 2234.5, 2234.75, 2234.75, 2234.5, 2234.0, 2233.75, 2233.75, 2233.75, 2233.5], 'volume': [1410.0, 211.0, 263.0, 145.0, 240.0, 311.0, 254.0, 40.0, 478.0, 81.0, 73.0, 367.0, 764.0, 49.0, 185.0, 201.0, 8.0, 20.0, 67.0, 98.0, 157.0, 97.0, 14.0, 225.0, 246.0, 186.0, 36.0, 81.0, 258.0, 686.0]})}])}])}]